<a href="https://colab.research.google.com/github/PondKann/CXR-Project/blob/main/DenseNet201_Finetuning_BS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###### import

In [1]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np
import pandas as pd
from torchsummary import summary
from skimage.io import imread, imsave
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from imutils import paths
import argparse

from sklearn.model_selection import KFold
from sklearn import metrics
from scipy.stats import zscore

In [3]:
import pickle
import joblib

######Load data 1

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
data_dir = '/content/drive/My Drive/Senior Project/CXR Image3class'

train 3 class

In [15]:
Normal151_dir = os.path.join(data_dir,'Normal151')
TB150_dir = os.path.join(data_dir,'TB150')
CA150_dir = os.path.join(data_dir,'CA150')

All_3class_dir = os.path.join(data_dir,'All_3class')

######Load data 2

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
data_dir = '/content/drive/My Drive/Senior Project/CXR Image3class'

train 3 class

In [27]:
Normal151_dir = os.path.join(data_dir,'Normal151')
TB150_dir = os.path.join(data_dir,'TB150')
CA150_dir = os.path.join(data_dir,'CA150')

All_3class_dir = os.path.join(data_dir,'All_3class')

In [ ]:
os.listdir(Normal151_dir)

#### train 3 class  --- สร้างตารางเก็บ ชื่อภาพ และชื่อคลาส
ข้อมูลใน All_3class_dir รวมภาพทั้ง 3 คลาส รวม 451 ภาพ TB150, Normal151 และ CA150 (CA คือ lungcancer)

In [7]:
Normal151_dir = os.path.join(data_dir,'Normal151')
TB150_dir = os.path.join(data_dir,'TB150')
CA150_dir = os.path.join(data_dir,'CA150')

All_3class_dir = os.path.join(data_dir,'All_3class')

In [8]:
normal = []
for i in range(len(os.listdir(Normal151_dir))):
    img = os.listdir(Normal151_dir)[i]
    normal.append(img)
    
tnormal = pd.DataFrame({'ImageName':[normal][0],
                                       'NameType': "Normal" ,
                                       'NumberType': 0                      })

In [9]:
tuberculosis = []
for i in range(len(os.listdir(TB150_dir))):
    img = os.listdir(TB150_dir)[i]
    tuberculosis.append(img)

ttuberculosis = pd.DataFrame({'ImageName':[tuberculosis][0],
                                       'NameType': "Tuberculosis" ,
                                       'NumberType': 1                        })

In [10]:
lungcancer = []
for i in range(len(os.listdir(CA150_dir))):
    img = os.listdir(CA150_dir)[i]
    lungcancer.append(img)

tlungcancer = pd.DataFrame({'ImageName':[lungcancer][0],
                                       'NameType': "Lungcancer" ,
                                       'NumberType': 2                        })

In [11]:
data_train3 = pd.concat([tnormal, ttuberculosis, tlungcancer])
data_train3[-3:]

,ImageName,NameType,NumberType
147,ca26.jpg,Lungcancer,2
148,ca7.jpg,Lungcancer,2
149,ca8.jpg,Lungcancer,2


####  test 3 class จากเน็ต

In [ ]:
TB700_dir = os.path.join(data_dir,'test_model/Tuberculosis700')
Normal94_dir = os.path.join(data_dir,'test_model/NORMAL94')
CA150_dir = os.path.join(data_dir,'CA150')                  #อันเดิมที่ใช้เทรน

AllTest_2class = os.path.join(data_dir,'test_model/AllTest_2class')

In [ ]:
#สำหรับ test model
normal = []
for i in range(len(os.listdir(Normal94_dir))):
    img = os.listdir(Normal94_dir)[i]
    normal.append(img)
    
tnormal94 = pd.DataFrame({'ImageName':[normal][0],
                                       'NameType': "Normal" ,
                                       'NumberType': 0                      })

In [ ]:
#สำหรับ test model
tb700 = []
for i in range(len(os.listdir(TB700_dir))):
    img = os.listdir(TB700_dir)[i]
    tb700.append(img)

ttb700 = pd.DataFrame({'ImageName':[tb700][0],
                                       'NameType': "Tuberculosis" ,
                                       'NumberType': 1                        })

In [ ]:
#สำหรับ test model ข้อม฿ลเดิม
lungcancer = []
for i in range(len(os.listdir(CA150_dir))):
    img = os.listdir(CA150_dir)[i]
    lungcancer.append(img)

tlungcancer = pd.DataFrame({'ImageName':[lungcancer][0],
                                       'NameType': "Lungcancer" ,
                                       'NumberType': 2                        })

In [ ]:
data_test2 = pd.concat([tnormal94[:30], ttb700[:30], tlungcancer[:10]])

In [ ]:
test_NorTB = []
for ImageName in tqdm(data_test2['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test2
    image_path = AllTest_2class +'/' + ImageName                          # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (299, 299))
    test_NorTB.append(img)

100%|██████████| 70/70 [00:19<00:00,  3.60it/s]


In [ ]:
data_test = np.array(test_NorTB, dtype="float32") / 255.0

labels_test = np.array(data_test2[['NameType']] )        
labels_test.shape = (len(labels_test),) 

le3_Incep = LabelEncoder()                                                                     
labels_test = le3_Incep.fit_transform(labels_test)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels_test = to_categorical(labels_test, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

# Fine tuning 0-600

## 8) K-fold  -- DenseNet201   

### -->ACC = 0.70
### -->เวลา 5384.864 วินาที

*   BS = 1
*   fix weight & bias in layer 0-600
*   activation="softmax"
*   learning_rate=0.0001
*   epoch 30

In [ ]:
train_img_Dense = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (224, 224))                                       #ขนาดของ input
    train_img_Dense.append(img)

data_Dense = np.array(train_img_Dense, dtype="float32") / 255.0

100%|██████████| 451/451 [00:04<00:00, 91.90it/s] 


In [ ]:
labels_Dense = np.array(data_train3[['NameType']] )        
labels_Dense.shape = (len(labels_Dense),) 

le_Dense = LabelEncoder()                                                                     
labels_Dense = le_Dense.fit_transform(labels_Dense)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels_Dense = to_categorical(labels_Dense, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

In [ ]:
INIT_LR = 1e-1
BS = 1
EPOCHS = 50

aug8 = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.1,
		height_shift_range=0.1,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
base_model_Dense = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
len(base_model_Dense.layers)

707

In [ ]:
base_model_Dense.trainable = True
for layer in base_model_Dense.layers[:600]:    #fix w & bias in layer 0-600
    layer.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Dense.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Dense8 = tf.keras.models.Model(inputs=base_model_Dense.input, outputs=prediction_layer)

model_Dense8.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
kf_Dense8 = KFold(5, shuffle=True, random_state=40) 

oos_y8 = []
oos_pred8 = []
fold = 0

for train, test in kf_Dense8.split(data_Dense):
    fold+=1
    print(f"Fold #{fold}")
        
    x_train_Dense8 = data_Dense[train]
    y_train_Dense8 = labels_Dense[train]
    x_test_Dense8 = data_Dense[test]
    y_test_Dense8 = labels_Dense[test]

    H_Dense8 = model_Dense8.fit( x=aug8.flow(x_train_Dense8, y_train_Dense8, batch_size=BS),
	                                            validation_data=(x_test_Dense8, y_test_Dense8),
	                                            steps_per_epoch=len(x_train_Dense8) // BS,
	                                            epochs=30,  callbacks = [callback])
    
    pred_Dense8 = model_Dense8.predict(x_test_Dense8)
    
    oos_y8.append(y_test_Dense8)
    oos_pred8.append(pred_Dense8)  
    print(classification_report(y_test_Dense8.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Dense8.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

#   score = metrics.accuracy_score(y_test3_Incep.argmax(axis=1),	        #ค่า acc
#                                               pred3_Incep.argmax(axis=1)
#    print(f"Fold score (accuracy): {score}")

#    print(confusion_matrix(testY3_Incep.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_Incep.argmax(axis=1) ) )

Fold #1
Epoch 1/30
360/360 [==============================] - 150s 374ms/step - loss: 0.6440 - accuracy: 0.3694 - val_loss: 0.5706 - val_accuracy: 0.5934
Epoch 2/30
360/360 [==============================] - 143s 396ms/step - loss: 0.5860 - accuracy: 0.5083 - val_loss: 0.5384 - val_accuracy: 0.5385
Epoch 3/30
360/360 [==============================] - 145s 403ms/step - loss: 0.5471 - accuracy: 0.6139 - val_loss: 0.5017 - val_accuracy: 0.6593
Epoch 4/30
360/360 [==============================] - 146s 406ms/step - loss: 0.5031 - accuracy: 0.6611 - val_loss: 0.5202 - val_accuracy: 0.5714
Epoch 5/30
360/360 [==============================] - 145s 403ms/step - loss: 0.4556 - accuracy: 0.7139 - val_loss: 0.4772 - val_accuracy: 0.6484
Epoch 6/30
360/360 [==============================] - 147s 409ms/step - loss: 0.4224 - accuracy: 0.7000 - val_loss: 0.4539 - val_accuracy: 0.7253
Epoch 7/30
360/360 [==============================] - 145s 403ms/step - loss: 0.3967 - accuracy: 0.7722 - val_loss: 

In [ ]:
oos_y8 = np.concatenate(oos_y8)
oos_pred8 = np.concatenate(oos_pred8)

In [ ]:
score8 = metrics.accuracy_score(oos_y8.argmax(axis=1),	        #ค่า acc
                                             oos_pred8.argmax(axis=1) )
print(f"Fold score (accuracy): {score8}")

print(classification_report(oos_y8.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        oos_pred8.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

Fold score (accuracy): 0.6962305986696231
              precision    recall  f1-score   support

  Lungcancer       0.83      0.64      0.72       150
      Normal       1.00      0.56      0.71       151
Tuberculosis       0.53      0.89      0.67       150

    accuracy                           0.70       451
   macro avg       0.79      0.70      0.70       451
weighted avg       0.79      0.70      0.70       451



## 8.1) K-fold  -- DenseNet201   

### -->ACC = 0.74
### -->เวลา 6021.688  วินาที

*   BS = 1
*   fix weight & bias in layer 0-600
*   activation="sigmoid"
*   learning_rate=0.0001
*   epoch 30

In [ ]:
train_img_Dense = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (224, 224))                                       #ขนาดของ input
    train_img_Dense.append(img)

data_Dense = np.array(train_img_Dense, dtype="float32") / 255.0

100%|██████████| 451/451 [00:35<00:00, 12.85it/s]


In [ ]:
labels_Dense = np.array(data_train3[['NameType']] )        
labels_Dense.shape = (len(labels_Dense),) 

le_Dense = LabelEncoder()                                                                     
labels_Dense = le_Dense.fit_transform(labels_Dense)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels_Dense = to_categorical(labels_Dense, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

In [ ]:
INIT_LR = 1e-1
BS = 1
EPOCHS = 50

aug81 = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.1,
		height_shift_range=0.1,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
base_model_Dense = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
len(base_model_Dense.layers)

74850304/74836368 [==============================] - 1s 0us/step


707

In [ ]:
base_model_Dense.trainable = True
for layer in base_model_Dense.layers[:600]:    #fix w & bias in layer 0-600
    layer.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Dense.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="sigmoid")(average_pooling_layer)   # unit 3
model_Dense81 = tf.keras.models.Model(inputs=base_model_Dense.input, outputs=prediction_layer)

model_Dense81.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
kf_Dense81 = KFold(5, shuffle=True, random_state=40) 

oos_y81 = []
oos_pred81 = []
fold = 0

for train, test in kf_Dense81.split(data_Dense):
    fold+=1
    print(f"Fold #{fold}")
        
    x_train_Dense81 = data_Dense[train]
    y_train_Dense81 = labels_Dense[train]
    x_test_Dense81 = data_Dense[test]
    y_test_Dense81 = labels_Dense[test]

    H_Dense81 = model_Dense81.fit( x=aug81.flow(x_train_Dense81, y_train_Dense81, batch_size=BS),
	                                            validation_data=(x_test_Dense81, y_test_Dense81),
	                                            steps_per_epoch=len(x_train_Dense81) // BS,
	                                            epochs=30,  callbacks = [callback])
    
    pred_Dense81 = model_Dense81.predict(x_test_Dense81)
    
    oos_y81.append(y_test_Dense81)
    oos_pred81.append(pred_Dense81)  
    print(classification_report(y_test_Dense81.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Dense81.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

#   score = metrics.accuracy_score(y_test3_Incep.argmax(axis=1),	        #ค่า acc
#                                               pred3_Incep.argmax(axis=1)
#    print(f"Fold score (accuracy): {score}")

#    print(confusion_matrix(testY3_Incep.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_Incep.argmax(axis=1) ) )

Fold #1
Epoch 1/30
360/360 [==============================] - 189s 444ms/step - loss: 0.6305 - accuracy: 0.3889 - val_loss: 0.5381 - val_accuracy: 0.6593
Epoch 2/30
360/360 [==============================] - 205s 568ms/step - loss: 0.5927 - accuracy: 0.5250 - val_loss: 0.5139 - val_accuracy: 0.6813
Epoch 3/30
360/360 [==============================] - 222s 618ms/step - loss: 0.5457 - accuracy: 0.6472 - val_loss: 0.5171 - val_accuracy: 0.6813
Epoch 4/30
360/360 [==============================] - 265s 737ms/step - loss: 0.4964 - accuracy: 0.6694 - val_loss: 0.5114 - val_accuracy: 0.6484
Epoch 5/30
360/360 [==============================] - 272s 756ms/step - loss: 0.4598 - accuracy: 0.7028 - val_loss: 0.4777 - val_accuracy: 0.6484
              precision    recall  f1-score   support

  Lungcancer       0.67      0.07      0.13        28
      Normal       0.86      0.97      0.91        33
Tuberculosis       0.49      0.83      0.62        30

    accuracy                           0.65 

In [ ]:
oos_y81 = np.concatenate(oos_y81)
oos_pred81 = np.concatenate(oos_pred81)

In [ ]:
score81 = metrics.accuracy_score(oos_y81.argmax(axis=1),	        #ค่า acc
                                             oos_pred81.argmax(axis=1) )
print(f"Fold score (accuracy): {score81}")

print(classification_report(oos_y81.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        oos_pred81.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

Fold score (accuracy): 0.738359201773836
              precision    recall  f1-score   support

  Lungcancer       0.83      0.55      0.66       150
      Normal       0.94      0.79      0.86       151
Tuberculosis       0.58      0.87      0.70       150

    accuracy                           0.74       451
   macro avg       0.79      0.74      0.74       451
weighted avg       0.79      0.74      0.74       451



## 1) K-fold  -- DenseNet201

### -->ACC =  0.88
### -->เวลา 3636 วินาที

*   BS = 8
*   fix weight & bias in layer 0-600
*   activation="softmax"
*   learning_rate=0.0001
*   epoch 30

In [ ]:
train_img_Dense = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (224, 224))                                       #ขนาดของ input
    train_img_Dense.append(img)

data_Dense = np.array(train_img_Dense, dtype="float32") / 255.0

100%|██████████| 451/451 [02:20<00:00,  3.22it/s]


In [ ]:
labels_Dense = np.array(data_train3[['NameType']] )        
labels_Dense.shape = (len(labels_Dense),) 

le_Dense = LabelEncoder()                                                                     
labels_Dense = le_Dense.fit_transform(labels_Dense)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels_Dense = to_categorical(labels_Dense, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

In [ ]:
INIT_LR = 1e-1
BS = 8
EPOCHS = 50

aug = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.1,
		height_shift_range=0.1,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
base_model_Dense = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
len(base_model_Dense.layers)

74850304/74836368 [==============================] - 1s 0us/step


707

In [ ]:
base_model_Dense.trainable = True
for layer in base_model_Dense.layers[:600]:    #fix w & bias in layer 0-600
    layer.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Dense.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Dense = tf.keras.models.Model(inputs=base_model_Dense.input, outputs=prediction_layer)

model_Dense.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
kf_Dense = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Dense.split(data_Dense):
    fold+=1
    print(f"Fold #{fold}")
        
    x_train_Dense = data_Dense[train]
    y_train_Dense = labels_Dense[train]
    x_test_Dense = data_Dense[test]
    y_test_Dense = labels_Dense[test]

    H_Dense = model_Dense.fit( x=aug.flow(x_train_Dense, y_train_Dense, batch_size=BS),
	                                            validation_data=(x_test_Dense, y_test_Dense),
	                                            steps_per_epoch=len(x_train_Dense) // BS,
	                                            epochs=30,  callbacks = [callback])
    
    pred_Dense = model_Dense.predict(x_test_Dense)
    
    oos_y.append(y_test_Dense)
    oos_pred.append(pred_Dense)  
    print(classification_report(y_test_Dense.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Dense.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

#   score = metrics.accuracy_score(y_test3_Incep.argmax(axis=1),	        #ค่า acc
#                                               pred3_Incep.argmax(axis=1)
#    print(f"Fold score (accuracy): {score}")

#    print(confusion_matrix(testY3_Incep.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_Incep.argmax(axis=1) ) )

Fold #1
Epoch 1/30
45/45 [==============================] - 136s 3s/step - loss: 0.5616 - accuracy: 0.5389 - val_loss: 0.4637 - val_accuracy: 0.7363
Epoch 2/30
45/45 [==============================] - 127s 3s/step - loss: 0.4063 - accuracy: 0.7417 - val_loss: 0.4005 - val_accuracy: 0.7582
Epoch 3/30
45/45 [==============================] - 130s 3s/step - loss: 0.3337 - accuracy: 0.8028 - val_loss: 0.3845 - val_accuracy: 0.7033
Epoch 4/30
45/45 [==============================] - 132s 3s/step - loss: 0.2868 - accuracy: 0.8417 - val_loss: 0.3558 - val_accuracy: 0.7363
Epoch 5/30
45/45 [==============================] - 132s 3s/step - loss: 0.2659 - accuracy: 0.8333 - val_loss: 0.3602 - val_accuracy: 0.7473
              precision    recall  f1-score   support

  Lungcancer       0.76      0.46      0.58        28
      Normal       0.89      0.94      0.91        33
Tuberculosis       0.62      0.80      0.70        30

    accuracy                           0.75        91
   macro avg   

In [ ]:
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)

In [ ]:
score = metrics.accuracy_score(oos_y.argmax(axis=1),	        #ค่า acc
                                             oos_pred.argmax(axis=1) )
print(f"Fold score (accuracy): {score}")

print(classification_report(oos_y.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        oos_pred.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

Fold score (accuracy): 0.8824833702882483
              precision    recall  f1-score   support

  Lungcancer       0.95      0.73      0.82       150
      Normal       0.97      0.98      0.97       151
Tuberculosis       0.77      0.94      0.85       150

    accuracy                           0.88       451
   macro avg       0.90      0.88      0.88       451
weighted avg       0.90      0.88      0.88       451



## 1.1) K-fold  -- DenseNet201  

### -->ACC = 0.92
### -->เวลา 4525.54 วินาที

*   BS = 8
*   fix weight & bias in layer 0-600
*   activation="sigmoid"
*   learning_rate=0.0001
*   epoch 30

In [ ]:
train_img_Dense = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (224, 224))                                       #ขนาดของ input
    train_img_Dense.append(img)

data_Dense = np.array(train_img_Dense, dtype="float32") / 255.0

100%|██████████| 451/451 [02:18<00:00,  3.26it/s]


In [ ]:
labels_Dense = np.array(data_train3[['NameType']] )        
labels_Dense.shape = (len(labels_Dense),) 

le_Dense = LabelEncoder()                                                                     
labels_Dense = le_Dense.fit_transform(labels_Dense)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels_Dense = to_categorical(labels_Dense, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

In [ ]:
INIT_LR = 1e-1
BS8 = 8
EPOCHS = 50

aug11 = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.1,
		height_shift_range=0.1,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
base_model_Dense = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
len(base_model_Dense.layers)

74850304/74836368 [==============================] - 0s 0us/step


707

In [ ]:
base_model_Dense.trainable = True
for layer in base_model_Dense.layers[:600]:    #fix w & bias in layer 0-600
    layer.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Dense.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Dense11 = tf.keras.models.Model(inputs=base_model_Dense.input, outputs=prediction_layer)

model_Dense11.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
kf_Dense11 = KFold(5, shuffle=True, random_state=40) 

oos_y11 = []
oos_pred11 = []
fold = 0

for train, test in kf_Dense11.split(data_Dense):
    fold+=1
    print(f"Fold #{fold}")
        
    x_train_Dense11 = data_Dense[train]
    y_train_Dense11 = labels_Dense[train]
    x_test_Dense11 = data_Dense[test]
    y_test_Dense11 = labels_Dense[test]

    H_Dense11 = model_Dense11.fit( x=aug11.flow(x_train_Dense11, y_train_Dense11, batch_size=BS8),
	                                            validation_data=(x_test_Dense11, y_test_Dense11 ),
	                                            steps_per_epoch=len(x_train_Dense11 ) // BS8,
	                                            epochs=30,  callbacks = [callback])
    
    pred_Dense11 = model_Dense11.predict(x_test_Dense11)
    
    oos_y11.append(y_test_Dense11)
    oos_pred11.append(pred_Dense11)  
    print(classification_report(y_test_Dense11.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Dense11.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

#   score = metrics.accuracy_score(y_test3_Incep.argmax(axis=1),	        #ค่า acc
#                                               pred3_Incep.argmax(axis=1)
#    print(f"Fold score (accuracy): {score}")

#    print(confusion_matrix(testY3_Incep.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_Incep.argmax(axis=1) ) )

Fold #1
Epoch 1/30
45/45 [==============================] - 148s 3s/step - loss: 0.5283 - accuracy: 0.5722 - val_loss: 0.4870 - val_accuracy: 0.6813
Epoch 2/30
45/45 [==============================] - 133s 3s/step - loss: 0.4103 - accuracy: 0.7389 - val_loss: 0.4323 - val_accuracy: 0.6484
Epoch 3/30
45/45 [==============================] - 135s 3s/step - loss: 0.3447 - accuracy: 0.7972 - val_loss: 0.3855 - val_accuracy: 0.6703
Epoch 4/30
45/45 [==============================] - 135s 3s/step - loss: 0.3003 - accuracy: 0.8556 - val_loss: 0.3478 - val_accuracy: 0.7363
Epoch 5/30
45/45 [==============================] - 140s 3s/step - loss: 0.2853 - accuracy: 0.8528 - val_loss: 0.3571 - val_accuracy: 0.6923
Epoch 6/30
45/45 [==============================] - 135s 3s/step - loss: 0.2434 - accuracy: 0.8861 - val_loss: 0.3441 - val_accuracy: 0.7143
Epoch 7/30
45/45 [==============================] - 134s 3s/step - loss: 0.2325 - accuracy: 0.8833 - val_loss: 0.2945 - val_accuracy: 0.7802
Epoch

In [ ]:
oos_y11 = np.concatenate(oos_y11)
oos_pred11 = np.concatenate(oos_pred11)

In [ ]:
score11 = metrics.accuracy_score(oos_y11.argmax(axis=1),	        #ค่า acc
                                             oos_pred11.argmax(axis=1) )
print(f"Fold score (accuracy): {score11}")

print(classification_report(oos_y11.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        oos_pred11.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

Fold score (accuracy): 0.917960088691796
              precision    recall  f1-score   support

  Lungcancer       0.97      0.81      0.88       150
      Normal       0.99      0.98      0.98       151
Tuberculosis       0.82      0.97      0.89       150

    accuracy                           0.92       451
   macro avg       0.93      0.92      0.92       451
weighted avg       0.93      0.92      0.92       451



## 2) K-fold  -- DenseNet201

### -->ACC = 0.87
### -->เวลา 2959 นาที

*   BS = 16
*   fix weight & bias in layer 0-600
*   activation="softmax"
*   learning_rate=0.0001
*   epoch 30

In [ ]:
train_img_Dense = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (224, 224))                                       #ขนาดของ input
    train_img_Dense.append(img)

data_Dense = np.array(train_img_Dense, dtype="float32") / 255.0

100%|██████████| 451/451 [00:05<00:00, 83.30it/s] 


In [ ]:
labels_Dense = np.array(data_train3[['NameType']] )        
labels_Dense.shape = (len(labels_Dense),) 

le_Dense = LabelEncoder()                                                                     
labels_Dense = le_Dense.fit_transform(labels_Dense)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels_Dense = to_categorical(labels_Dense, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

In [ ]:
INIT_LR = 1e-1
BS16 = 16
EPOCHS = 50

aug2 = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.1,
		height_shift_range=0.1,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
base_model_Dense = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
len(base_model_Dense.layers)

707

In [ ]:
base_model_Dense.trainable = True
for layer in base_model_Dense.layers[:600]:    #fix w & bias in layer 0-600
    layer.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Dense.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Dense2 = tf.keras.models.Model(inputs=base_model_Dense.input, outputs=prediction_layer)

model_Dense2.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
kf_Dense2 = KFold(5, shuffle=True, random_state=40) 

oos_y2 = []
oos_pred2 = []
fold = 0

for train, test in kf_Dense2.split(data_Dense):
    fold+=1
    print(f"Fold #{fold}")
        
    x_train_Dense2 = data_Dense[train]
    y_train_Dense2 = labels_Dense[train]
    x_test_Dense2 = data_Dense[test]
    y_test_Dense2 = labels_Dense[test]

    H_Dense2 = model_Dense2.fit( x=aug2.flow(x_train_Dense2, y_train_Dense2, batch_size=BS16),
	                                            validation_data=(x_test_Dense2, y_test_Dense2),
	                                            steps_per_epoch=len(x_train_Dense2) // BS16,
	                                            epochs=30,  callbacks = [callback])
    
    pred_Dense2 = model_Dense2.predict(x_test_Dense2)
    
    oos_y2.append(y_test_Dense2)
    oos_pred2.append(pred_Dense2)  
    print(classification_report(y_test_Dense2.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Dense2.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

#   score = metrics.accuracy_score(y_test3_Incep.argmax(axis=1),	        #ค่า acc
#                                               pred3_Incep.argmax(axis=1)
#    print(f"Fold score (accuracy): {score}")

#    print(confusion_matrix(testY3_Incep.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_Incep.argmax(axis=1) ) )

Fold #1
Epoch 1/30
22/22 [==============================] - 135s 5s/step - loss: 0.5737 - accuracy: 0.5581 - val_loss: 0.5526 - val_accuracy: 0.6154
Epoch 2/30
22/22 [==============================] - 116s 5s/step - loss: 0.4061 - accuracy: 0.7645 - val_loss: 0.4775 - val_accuracy: 0.6484
Epoch 3/30
22/22 [==============================] - 116s 5s/step - loss: 0.3414 - accuracy: 0.8052 - val_loss: 0.4230 - val_accuracy: 0.6923
Epoch 4/30
22/22 [==============================] - 115s 5s/step - loss: 0.3156 - accuracy: 0.8372 - val_loss: 0.3981 - val_accuracy: 0.6923
Epoch 5/30
22/22 [==============================] - 118s 5s/step - loss: 0.2713 - accuracy: 0.8523 - val_loss: 0.3834 - val_accuracy: 0.6923
Epoch 6/30
22/22 [==============================] - 116s 5s/step - loss: 0.2485 - accuracy: 0.8808 - val_loss: 0.4143 - val_accuracy: 0.6813
              precision    recall  f1-score   support

  Lungcancer       0.75      0.21      0.33        28
      Normal       0.82      0.94    

In [ ]:
oos_y2 = np.concatenate(oos_y2)
oos_pred2 = np.concatenate(oos_pred2)

In [ ]:
score = metrics.accuracy_score(oos_y2.argmax(axis=1),	        #ค่า acc
                                             oos_pred2.argmax(axis=1) )
print(f"Fold score (accuracy): {score}")

print(classification_report(oos_y2.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        oos_pred2.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

Fold score (accuracy): 0.8669623059866962
              precision    recall  f1-score   support

  Lungcancer       0.97      0.67      0.80       150
      Normal       0.94      0.97      0.96       151
Tuberculosis       0.75      0.95      0.84       150

    accuracy                           0.87       451
   macro avg       0.89      0.87      0.86       451
weighted avg       0.89      0.87      0.86       451



## 2.1) K-fold  -- DenseNet201  

### -->ACC = 0.88
### -->เวลา 4005.763 นาที

*   BS = 16
*   fix weight & bias in layer 0-600
*   activation="sigmoid"
*   learning_rate=0.0001
*   epoch 30

In [ ]:
train_img_Dense = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (224, 224))                                       #ขนาดของ input
    train_img_Dense.append(img)

data_Dense = np.array(train_img_Dense, dtype="float32") / 255.0

100%|██████████| 451/451 [00:05<00:00, 80.41it/s] 


In [ ]:
labels_Dense = np.array(data_train3[['NameType']] )        
labels_Dense.shape = (len(labels_Dense),) 

le_Dense = LabelEncoder()                                                                     
labels_Dense = le_Dense.fit_transform(labels_Dense)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels_Dense = to_categorical(labels_Dense, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

In [ ]:
INIT_LR = 1e-1
BS16 = 16
EPOCHS = 50

aug21 = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.1,
		height_shift_range=0.1,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
base_model_Dense = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
len(base_model_Dense.layers)

707

In [ ]:
base_model_Dense.trainable = True
for layer in base_model_Dense.layers[:600]:    #fix w & bias in layer 0-600
    layer.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Dense.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="sigmoid")(average_pooling_layer)   # unit 3
model_Dense21 = tf.keras.models.Model(inputs=base_model_Dense.input, outputs=prediction_layer)

model_Dense21.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
kf_Dense21 = KFold(5, shuffle=True, random_state=40) 

oos_y21 = []
oos_pred21 = []
fold = 0

for train, test in kf_Dense21.split(data_Dense):
    fold+=1
    print(f"Fold #{fold}")
        
    x_train_Dense21 = data_Dense[train]
    y_train_Dense21 = labels_Dense[train]
    x_test_Dense21 = data_Dense[test]
    y_test_Dense21 = labels_Dense[test]

    H_Dense21 = model_Dense21.fit( x=aug21.flow(x_train_Dense21, y_train_Dense21, batch_size=BS16),
	                                            validation_data=(x_test_Dense21, y_test_Dense21),
	                                            steps_per_epoch=len(x_train_Dense21) // BS16,
	                                            epochs=30,  callbacks = [callback])
    
    pred_Dense21 = model_Dense21.predict(x_test_Dense21)
    
    oos_y21.append(y_test_Dense21)
    oos_pred21.append(pred_Dense21)  
    print(classification_report(y_test_Dense21.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Dense21.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

#   score = metrics.accuracy_score(y_test3_Incep.argmax(axis=1),	        #ค่า acc
#                                               pred3_Incep.argmax(axis=1)
#    print(f"Fold score (accuracy): {score}")

#    print(confusion_matrix(testY3_Incep.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_Incep.argmax(axis=1) ) )

Fold #1
Epoch 1/30
22/22 [==============================] - 139s 6s/step - loss: 0.5372 - accuracy: 0.6453 - val_loss: 0.5158 - val_accuracy: 0.6703
Epoch 2/30
22/22 [==============================] - 115s 5s/step - loss: 0.3991 - accuracy: 0.7558 - val_loss: 0.4614 - val_accuracy: 0.6923
Epoch 3/30
22/22 [==============================] - 115s 5s/step - loss: 0.3544 - accuracy: 0.8140 - val_loss: 0.4204 - val_accuracy: 0.6703
Epoch 4/30
22/22 [==============================] - 117s 5s/step - loss: 0.3118 - accuracy: 0.8081 - val_loss: 0.3988 - val_accuracy: 0.6813
Epoch 5/30
22/22 [==============================] - 117s 5s/step - loss: 0.2641 - accuracy: 0.8692 - val_loss: 0.3720 - val_accuracy: 0.7143
Epoch 6/30
22/22 [==============================] - 117s 5s/step - loss: 0.2643 - accuracy: 0.8605 - val_loss: 0.3792 - val_accuracy: 0.7033
Epoch 7/30
22/22 [==============================] - 117s 5s/step - loss: 0.2244 - accuracy: 0.8895 - val_loss: 0.3872 - val_accuracy: 0.6923
Epoch

In [ ]:
oos_y21 = np.concatenate(oos_y21)
oos_pred21 = np.concatenate(oos_pred21)

In [ ]:
score21 = metrics.accuracy_score(oos_y21.argmax(axis=1),	        #ค่า acc
                                             oos_pred21.argmax(axis=1) )
print(f"Fold score (accuracy): {score21}")

print(classification_report(oos_y21.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        oos_pred21.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

Fold score (accuracy): 0.8758314855875832
              precision    recall  f1-score   support

  Lungcancer       0.97      0.69      0.81       150
      Normal       0.98      0.97      0.97       151
Tuberculosis       0.74      0.97      0.84       150

    accuracy                           0.88       451
   macro avg       0.90      0.88      0.87       451
weighted avg       0.90      0.88      0.87       451



## 3) K-fold  -- DenseNet201

### -->ACC =  0.89
### -->เวลา 3762 วินาที

*   BS = 32
*   fix weight & bias in layer 0-600
*   activation="softmax"
*   learning_rate=0.0001
*   epoch 30

In [ ]:
train_img_Dense = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (224, 224))                                       #ขนาดของ input
    train_img_Dense.append(img)

data_Dense = np.array(train_img_Dense, dtype="float32") / 255.0

100%|██████████| 451/451 [00:05<00:00, 82.46it/s] 


In [ ]:
labels_Dense = np.array(data_train3[['NameType']] )        
labels_Dense.shape = (len(labels_Dense),) 

le_Dense = LabelEncoder()                                                                     
labels_Dense = le_Dense.fit_transform(labels_Dense)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels_Dense = to_categorical(labels_Dense, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

In [ ]:
INIT_LR = 1e-1
BS32 = 32
EPOCHS = 50

aug3 = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.1,
		height_shift_range=0.1,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
base_model_Dense = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
len(base_model_Dense.layers)

707

In [ ]:
base_model_Dense.trainable = True
for layer in base_model_Dense.layers[:600]:    #fix w & bias in layer 0-600
    layer.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Dense.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Dense3 = tf.keras.models.Model(inputs=base_model_Dense.input, outputs=prediction_layer)

model_Dense3.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
kf_Dense3 = KFold(5, shuffle=True, random_state=40) 

oos_y3 = []
oos_pred3 = []
fold = 0

for train, test in kf_Dense3.split(data_Dense):
    fold+=1
    print(f"Fold #{fold}")
        
    x_train_Dense3 = data_Dense[train]
    y_train_Dense3 = labels_Dense[train]
    x_test_Dense3 = data_Dense[test]
    y_test_Dense3 = labels_Dense[test]

    H_Dense3 = model_Dense3.fit( x=aug3.flow(x_train_Dense3, y_train_Dense3, batch_size=BS32),
	                                            validation_data=(x_test_Dense3, y_test_Dense3),
	                                            steps_per_epoch=len(x_train_Dense3) // BS32,
	                                            epochs=30,  callbacks = [callback])
    
    pred_Dense3 = model_Dense3.predict(x_test_Dense3)
    
    oos_y3.append(y_test_Dense3)
    oos_pred3.append(pred_Dense3)  
    print(classification_report(y_test_Dense3.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Dense3.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

#   score = metrics.accuracy_score(y_test3_Incep.argmax(axis=1),	        #ค่า acc
#                                               pred3_Incep.argmax(axis=1)
#    print(f"Fold score (accuracy): {score}")

#    print(confusion_matrix(testY3_Incep.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_Incep.argmax(axis=1) ) )

Fold #1
Epoch 1/30
11/11 [==============================] - 127s 10s/step - loss: 0.5186 - accuracy: 0.6463 - val_loss: 0.4981 - val_accuracy: 0.6374
Epoch 2/30
11/11 [==============================] - 107s 10s/step - loss: 0.4018 - accuracy: 0.7409 - val_loss: 0.4807 - val_accuracy: 0.6154
Epoch 3/30
11/11 [==============================] - 109s 10s/step - loss: 0.3581 - accuracy: 0.7957 - val_loss: 0.4455 - val_accuracy: 0.6813
Epoch 4/30
11/11 [==============================] - 108s 10s/step - loss: 0.3404 - accuracy: 0.8079 - val_loss: 0.4243 - val_accuracy: 0.7143
Epoch 5/30
11/11 [==============================] - 108s 10s/step - loss: 0.2998 - accuracy: 0.8598 - val_loss: 0.3962 - val_accuracy: 0.7363
Epoch 6/30
11/11 [==============================] - 107s 10s/step - loss: 0.2769 - accuracy: 0.8750 - val_loss: 0.3835 - val_accuracy: 0.7363
Epoch 7/30
11/11 [==============================] - 114s 10s/step - loss: 0.2660 - accuracy: 0.8628 - val_loss: 0.3695 - val_accuracy: 0.758

In [ ]:
oos_y3 = np.concatenate(oos_y3)
oos_pred3 = np.concatenate(oos_pred3)

In [ ]:
score = metrics.accuracy_score(oos_y3.argmax(axis=1),	        #ค่า acc
                                             oos_pred3.argmax(axis=1) )
print(f"Fold score (accuracy): {score}")

print(classification_report(oos_y3.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        oos_pred3.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

Fold score (accuracy): 0.8869179600886918
              precision    recall  f1-score   support

  Lungcancer       0.97      0.75      0.84       150
      Normal       0.91      0.99      0.95       151
Tuberculosis       0.81      0.93      0.86       150

    accuracy                           0.89       451
   macro avg       0.90      0.89      0.88       451
weighted avg       0.90      0.89      0.88       451



## 3.1) K-fold  -- DenseNet201

### -->ACC = 0.86
### -->เวลา 2582 วินาที

*   BS = 32
*   fix weight & bias in layer 0-600
*   activation="sigmoid"
*   learning_rate=0.0001
*   epoch 30

In [ ]:
train_img_Dense = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (224, 224))                                       #ขนาดของ input
    train_img_Dense.append(img)

data_Dense = np.array(train_img_Dense, dtype="float32") / 255.0

100%|██████████| 451/451 [04:08<00:00,  1.81it/s]


In [ ]:
labels_Dense = np.array(data_train3[['NameType']] )        
labels_Dense.shape = (len(labels_Dense),) 

le_Dense = LabelEncoder()                                                                     
labels_Dense = le_Dense.fit_transform(labels_Dense)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels_Dense = to_categorical(labels_Dense, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

In [ ]:
INIT_LR = 1e-1
BS32 = 32
EPOCHS = 50

aug31 = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.1,
		height_shift_range=0.1,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
base_model_Dense = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
len(base_model_Dense.layers)

74850304/74836368 [==============================] - 1s 0us/step


707

In [ ]:
base_model_Dense.trainable = True
for layer in base_model_Dense.layers[:600]:    #fix w & bias in layer 0-600
    layer.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Dense.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="sigmoid")(average_pooling_layer)   # unit 3
model_Dense31 = tf.keras.models.Model(inputs=base_model_Dense.input, outputs=prediction_layer)

model_Dense31.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
kf_Dense31 = KFold(5, shuffle=True, random_state=40) 

oos_y31 = []
oos_pred31 = []
fold = 0

for train, test in kf_Dense31.split(data_Dense):
    fold+=1
    print(f"Fold #{fold}")
        
    x_train_Dense31 = data_Dense[train]
    y_train_Dense31 = labels_Dense[train]
    x_test_Dense31 = data_Dense[test]
    y_test_Dense31 = labels_Dense[test]

    H_Dense31 = model_Dense31.fit( x=aug31.flow(x_train_Dense31, y_train_Dense31, batch_size=BS32),
	                                            validation_data=(x_test_Dense31, y_test_Dense31),
	                                            steps_per_epoch=len(x_train_Dense31) // BS32,
	                                            epochs=30,  callbacks = [callback])
    
    pred_Dense31 = model_Dense31.predict(x_test_Dense31)
    
    oos_y31.append(y_test_Dense31)
    oos_pred31.append(pred_Dense31)  
    print(classification_report(y_test_Dense31.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Dense31.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

#   score = metrics.accuracy_score(y_test3_Incep.argmax(axis=1),	        #ค่า acc
#                                               pred3_Incep.argmax(axis=1)
#    print(f"Fold score (accuracy): {score}")

#    print(confusion_matrix(testY3_Incep.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_Incep.argmax(axis=1) ) )

Fold #1
Epoch 1/30
11/11 [==============================] - 90s 7s/step - loss: 0.5900 - accuracy: 0.5305 - val_loss: 0.5063 - val_accuracy: 0.6923
Epoch 2/30
11/11 [==============================] - 75s 7s/step - loss: 0.4283 - accuracy: 0.7500 - val_loss: 0.4602 - val_accuracy: 0.7363
Epoch 3/30
11/11 [==============================] - 75s 7s/step - loss: 0.3727 - accuracy: 0.7744 - val_loss: 0.4248 - val_accuracy: 0.7473
Epoch 4/30
11/11 [==============================] - 75s 7s/step - loss: 0.3294 - accuracy: 0.8201 - val_loss: 0.4080 - val_accuracy: 0.7473
Epoch 5/30
11/11 [==============================] - 76s 7s/step - loss: 0.3032 - accuracy: 0.8354 - val_loss: 0.3939 - val_accuracy: 0.6923
Epoch 6/30
11/11 [==============================] - 76s 7s/step - loss: 0.2886 - accuracy: 0.8476 - val_loss: 0.3915 - val_accuracy: 0.6923
              precision    recall  f1-score   support

  Lungcancer       0.75      0.32      0.45        28
      Normal       0.75      1.00      0.86

In [ ]:
oos_y31 = np.concatenate(oos_y31)
oos_pred31 = np.concatenate(oos_pred31)

In [ ]:
score31 = metrics.accuracy_score(oos_y31.argmax(axis=1),	        #ค่า acc
                                             oos_pred31.argmax(axis=1) )
print(f"Fold score (accuracy): {score31}")

print(classification_report(oos_y31.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        oos_pred31.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

Fold score (accuracy): 0.8625277161862528
              precision    recall  f1-score   support

  Lungcancer       0.95      0.71      0.81       150
      Normal       0.87      1.00      0.93       151
Tuberculosis       0.79      0.88      0.83       150

    accuracy                           0.86       451
   macro avg       0.87      0.86      0.86       451
weighted avg       0.87      0.86      0.86       451



# Fine tuning 0-500

## 4) K-fold  -- DenseNet201

### -->ACC = 0.87
### -->เวลา - วินาที

*   BS = 8
*   fix weight & bias in layer 0-500
*   activation="softmax"
*   learning_rate=0.0001
*   epoch 30

In [ ]:
train_img_Dense = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (224, 224))                                       #ขนาดของ input
    train_img_Dense.append(img)

data_Dense = np.array(train_img_Dense, dtype="float32") / 255.0

In [19]:
labels_Dense = np.array(data_train3[['NameType']] )        
labels_Dense.shape = (len(labels_Dense),) 

le_Dense = LabelEncoder()                                                                     
labels_Dense = le_Dense.fit_transform(labels_Dense)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels_Dense = to_categorical(labels_Dense, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

In [20]:
INIT_LR = 1e-1
BS = 8
EPOCHS = 50

aug = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.1,
		height_shift_range=0.1,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [21]:
base_model_Dense = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
len(base_model_Dense.layers)

74850304/74836368 [==============================] - 1s 0us/step


707

In [22]:
base_model_Dense.trainable = True
for layer in base_model_Dense.layers[:500]:    #fix w & bias in layer 0-500
    layer.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Dense.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Dense = tf.keras.models.Model(inputs=base_model_Dense.input, outputs=prediction_layer)

model_Dense.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
kf_Dense = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Dense.split(data_Dense):
    fold+=1
    print(f"Fold #{fold}")
        
    x_train_Dense = data_Dense[train]
    y_train_Dense = labels_Dense[train]
    x_test_Dense = data_Dense[test]
    y_test_Dense = labels_Dense[test]

    H_Dense = model_Dense.fit( x=aug.flow(x_train_Dense, y_train_Dense, batch_size=BS),
	                                            validation_data=(x_test_Dense, y_test_Dense),
	                                            steps_per_epoch=len(x_train_Dense) // BS,
	                                            epochs=30,  callbacks = [callback])
    
    pred_Dense = model_Dense.predict(x_test_Dense)
    
    oos_y.append(y_test_Dense)
    oos_pred.append(pred_Dense)  
    print(classification_report(y_test_Dense.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Dense.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

#   score = metrics.accuracy_score(y_test3_Incep.argmax(axis=1),	        #ค่า acc
#                                               pred3_Incep.argmax(axis=1)
#    print(f"Fold score (accuracy): {score}")

#    print(confusion_matrix(testY3_Incep.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_Incep.argmax(axis=1) ) )

In [ ]:
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)

In [ ]:
score = metrics.accuracy_score(oos_y.argmax(axis=1),	        #ค่า acc
                                             oos_pred.argmax(axis=1) )
print(f"Fold score (accuracy): {score}")

print(classification_report(oos_y.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        oos_pred.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

## 4.1) K-fold  -- DenseNet201  

### -->ACC = 0.90
### -->เวลา 4575.96 วินาที

*   BS = 8
*   fix weight & bias in layer 0-500
*   activation="sigmoid"
*   learning_rate=0.0001
*   epoch 30

In [ ]:
train_img_Dense = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (224, 224))                                       #ขนาดของ input
    train_img_Dense.append(img)

data_Dense = np.array(train_img_Dense, dtype="float32") / 255.0

100%|██████████| 451/451 [00:05<00:00, 81.94it/s] 


In [ ]:
labels_Dense = np.array(data_train3[['NameType']] )        
labels_Dense.shape = (len(labels_Dense),) 

le_Dense = LabelEncoder()                                                                     
labels_Dense = le_Dense.fit_transform(labels_Dense)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels_Dense = to_categorical(labels_Dense, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

In [ ]:
INIT_LR = 1e-1
BS8 = 8
EPOCHS = 50

aug41 = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.1,
		height_shift_range=0.1,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
base_model_Dense = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
len(base_model_Dense.layers)

707

In [ ]:
base_model_Dense.trainable = True
for layer in base_model_Dense.layers[:500]:    #fix w & bias in layer 0-500
    layer.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Dense.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="sigmoid")(average_pooling_layer)   # unit 3
model_Dense41 = tf.keras.models.Model(inputs=base_model_Dense.input, outputs=prediction_layer)

model_Dense41.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
kf_Dense41 = KFold(5, shuffle=True, random_state=40) 

oos_y41 = []
oos_pred41 = []
fold = 0

for train, test in kf_Dense41.split(data_Dense):
    fold+=1
    print(f"Fold #{fold}")
        
    x_train_Dense41 = data_Dense[train]
    y_train_Dense41 = labels_Dense[train]
    x_test_Dense41 = data_Dense[test]
    y_test_Dense41 = labels_Dense[test]

    H_Dense41 = model_Dense41.fit( x=aug41.flow(x_train_Dense41, y_train_Dense41, batch_size=BS8),
	                                            validation_data=(x_test_Dense41, y_test_Dense41),
	                                            steps_per_epoch=len(x_train_Dense41) // BS8,
	                                            epochs=30,  callbacks = [callback])
    
    pred_Dense41 = model_Dense41.predict(x_test_Dense41)
    
    oos_y41.append(y_test_Dense41)
    oos_pred41.append(pred_Dense41)  
    print(classification_report(y_test_Dense41.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Dense41.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

#   score = metrics.accuracy_score(y_test3_Incep.argmax(axis=1),	        #ค่า acc
#                                               pred3_Incep.argmax(axis=1)
#    print(f"Fold score (accuracy): {score}")

#    print(confusion_matrix(testY3_Incep.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_Incep.argmax(axis=1) ) )

Fold #1
Epoch 1/30
45/45 [==============================] - 119s 2s/step - loss: 0.5008 - accuracy: 0.6444 - val_loss: 0.5943 - val_accuracy: 0.5604
Epoch 2/30
45/45 [==============================] - 99s 2s/step - loss: 0.3438 - accuracy: 0.7833 - val_loss: 0.5383 - val_accuracy: 0.6154
Epoch 3/30
45/45 [==============================] - 103s 2s/step - loss: 0.2788 - accuracy: 0.8528 - val_loss: 0.4846 - val_accuracy: 0.6154
Epoch 4/30
45/45 [==============================] - 104s 2s/step - loss: 0.2418 - accuracy: 0.8889 - val_loss: 0.4779 - val_accuracy: 0.6593
Epoch 5/30
45/45 [==============================] - 104s 2s/step - loss: 0.2206 - accuracy: 0.8750 - val_loss: 0.3742 - val_accuracy: 0.7253
Epoch 6/30
45/45 [==============================] - 105s 2s/step - loss: 0.1690 - accuracy: 0.9250 - val_loss: 0.5127 - val_accuracy: 0.7143
Epoch 7/30
45/45 [==============================] - 103s 2s/step - loss: 0.1680 - accuracy: 0.9306 - val_loss: 0.6544 - val_accuracy: 0.7143
Epoch 

In [ ]:
oos_y41 = np.concatenate(oos_y41)
oos_pred41 = np.concatenate(oos_pred41)

In [ ]:
score41 = metrics.accuracy_score(oos_y41.argmax(axis=1),	        #ค่า acc
                                             oos_pred41.argmax(axis=1) )
print(f"Fold score (accuracy): {score41}")

print(classification_report(oos_y41.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        oos_pred41.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

Fold score (accuracy): 0.8957871396895787
              precision    recall  f1-score   support

  Lungcancer       0.97      0.83      0.90       150
      Normal       0.99      0.87      0.93       151
Tuberculosis       0.78      0.98      0.87       150

    accuracy                           0.90       451
   macro avg       0.91      0.90      0.90       451
weighted avg       0.91      0.90      0.90       451



## 5) K-fold  -- DenseNet201

### -->ACC = 0.93
### -->เวลา - นาที

*   BS = 16
*   fix weight & bias in layer 0-500
*   activation="softmax"
*   learning_rate=0.0001
*   epoch 30

In [ ]:
train_img_Dense = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (224, 224))                                       #ขนาดของ input
    train_img_Dense.append(img)

data_Dense = np.array(train_img_Dense, dtype="float32") / 255.0

100%|██████████| 451/451 [00:05<00:00, 85.20it/s] 


In [ ]:
labels_Dense = np.array(data_train3[['NameType']] )        
labels_Dense.shape = (len(labels_Dense),) 

le_Dense = LabelEncoder()                                                                     
labels_Dense = le_Dense.fit_transform(labels_Dense)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels_Dense = to_categorical(labels_Dense, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

In [ ]:
INIT_LR = 1e-1
BS16 = 16
EPOCHS = 50

aug2 = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.1,
		height_shift_range=0.1,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
base_model_Dense = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
len(base_model_Dense.layers)

707

In [ ]:
base_model_Dense.trainable = True
for layer in base_model_Dense.layers[:600]:    #fix w & bias in layer 0-600
    layer.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Dense.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Dense2 = tf.keras.models.Model(inputs=base_model_Dense.input, outputs=prediction_layer)

model_Dense2.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
kf_Dense2 = KFold(5, shuffle=True, random_state=40) 

oos_y2 = []
oos_pred2 = []
fold = 0

for train, test in kf_Dense2.split(data_Dense):
    fold+=1
    print(f"Fold #{fold}")
        
    x_train_Dense2 = data_Dense[train]
    y_train_Dense2 = labels_Dense[train]
    x_test_Dense2 = data_Dense[test]
    y_test_Dense2 = labels_Dense[test]

    H_Dense2 = model_Dense2.fit( x=aug2.flow(x_train_Dense2, y_train_Dense2, batch_size=BS16),
	                                            validation_data=(x_test_Dense2, y_test_Dense2),
	                                            steps_per_epoch=len(x_train_Dense2) // BS16,
	                                            epochs=30,  callbacks = [callback])
    
    pred_Dense2 = model_Dense2.predict(x_test_Dense2)
    
    oos_y2.append(y_test_Dense2)
    oos_pred2.append(pred_Dense2)  
    print(classification_report(y_test_Dense2.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Dense2.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

#   score = metrics.accuracy_score(y_test3_Incep.argmax(axis=1),	        #ค่า acc
#                                               pred3_Incep.argmax(axis=1)
#    print(f"Fold score (accuracy): {score}")

#    print(confusion_matrix(testY3_Incep.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_Incep.argmax(axis=1) ) )

Fold #1
Epoch 1/30
22/22 [==============================] - 131s 5s/step - loss: 0.5335 - accuracy: 0.6570 - val_loss: 0.5158 - val_accuracy: 0.6593
Epoch 2/30
22/22 [==============================] - 114s 5s/step - loss: 0.3912 - accuracy: 0.7616 - val_loss: 0.4862 - val_accuracy: 0.6703
Epoch 3/30
22/22 [==============================] - 115s 5s/step - loss: 0.3135 - accuracy: 0.8227 - val_loss: 0.4457 - val_accuracy: 0.7033
Epoch 4/30
22/22 [==============================] - 114s 5s/step - loss: 0.2949 - accuracy: 0.8285 - val_loss: 0.4228 - val_accuracy: 0.6813
Epoch 5/30
22/22 [==============================] - 112s 5s/step - loss: 0.2592 - accuracy: 0.8750 - val_loss: 0.4161 - val_accuracy: 0.7033
Epoch 6/30
22/22 [==============================] - 112s 5s/step - loss: 0.2337 - accuracy: 0.8750 - val_loss: 0.3942 - val_accuracy: 0.7253
Epoch 7/30
22/22 [==============================] - 112s 5s/step - loss: 0.2114 - accuracy: 0.8866 - val_loss: 0.3773 - val_accuracy: 0.7363
Epoch

In [ ]:
oos_y2 = np.concatenate(oos_y2)
oos_pred2 = np.concatenate(oos_pred2)

In [ ]:
score = metrics.accuracy_score(oos_y2.argmax(axis=1),	        #ค่า acc
                                             oos_pred2.argmax(axis=1) )
print(f"Fold score (accuracy): {score}")

print(classification_report(oos_y2.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        oos_pred2.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

Fold score (accuracy): 0.926829268292683
              precision    recall  f1-score   support

  Lungcancer       0.98      0.83      0.90       150
      Normal       0.99      0.97      0.98       151
Tuberculosis       0.83      0.99      0.90       150

    accuracy                           0.93       451
   macro avg       0.94      0.93      0.93       451
weighted avg       0.94      0.93      0.93       451



## 5.1) K-fold  -- DenseNet201

### -->ACC = 0.93
### -->เวลา 4335.68 นาที

*   BS = 16
*   fix weight & bias in layer 0-500
*   activation="sigmoid"
*   learning_rate=0.0001
*   epoch 30

In [ ]:
train_img_Dense = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (224, 224))                                       #ขนาดของ input
    train_img_Dense.append(img)

data_Dense = np.array(train_img_Dense, dtype="float32") / 255.0

100%|██████████| 451/451 [00:05<00:00, 86.62it/s] 


In [ ]:
labels_Dense = np.array(data_train3[['NameType']] )        
labels_Dense.shape = (len(labels_Dense),) 

le_Dense = LabelEncoder()                                                                     
labels_Dense = le_Dense.fit_transform(labels_Dense)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels_Dense = to_categorical(labels_Dense, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

In [ ]:
INIT_LR = 1e-1
BS16 = 16
EPOCHS = 50

aug51 = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.1,
		height_shift_range=0.1,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
base_model_Dense = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
len(base_model_Dense.layers)

707

In [ ]:
base_model_Dense.trainable = True
for layer in base_model_Dense.layers[:500]:    #fix w & bias in layer 0-500
    layer.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Dense.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="sigmoid")(average_pooling_layer)   # unit 3
model_Dense51 = tf.keras.models.Model(inputs=base_model_Dense.input, outputs=prediction_layer)

model_Dense51.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
kf_Dense51 = KFold(5, shuffle=True, random_state=40) 

oos_y51 = []
oos_pred51 = []
fold = 0

for train, test in kf_Dense51.split(data_Dense):
    fold+=1
    print(f"Fold #{fold}")
        
    x_train_Dense51 = data_Dense[train]
    y_train_Dense51 = labels_Dense[train]
    x_test_Dense51 = data_Dense[test]
    y_test_Dense51 = labels_Dense[test]

    H_Dense51 = model_Dense51.fit( x=aug51.flow(x_train_Dense51, y_train_Dense51, batch_size=BS16),
	                                            validation_data=(x_test_Dense51, y_test_Dense51),
	                                            steps_per_epoch=len(x_train_Dense51) // BS16,
	                                            epochs=30,  callbacks = [callback])
    
    pred_Dense51 = model_Dense51.predict(x_test_Dense51)
    
    oos_y51.append(y_test_Dense51)
    oos_pred51.append(pred_Dense51)  
    print(classification_report(y_test_Dense51.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Dense51.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

#   score = metrics.accuracy_score(y_test3_Incep.argmax(axis=1),	        #ค่า acc
#                                               pred3_Incep.argmax(axis=1)
#    print(f"Fold score (accuracy): {score}")

#    print(confusion_matrix(testY3_Incep.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_Incep.argmax(axis=1) ) )

Fold #1
Epoch 1/30
22/22 [==============================] - 113s 4s/step - loss: 0.4910 - accuracy: 0.6818 - val_loss: 0.6110 - val_accuracy: 0.5495
Epoch 2/30
22/22 [==============================] - 91s 4s/step - loss: 0.3051 - accuracy: 0.8285 - val_loss: 0.6054 - val_accuracy: 0.5385
Epoch 3/30
22/22 [==============================] - 89s 4s/step - loss: 0.2485 - accuracy: 0.8692 - val_loss: 0.5594 - val_accuracy: 0.5714
Epoch 4/30
22/22 [==============================] - 89s 4s/step - loss: 0.2070 - accuracy: 0.8895 - val_loss: 0.4729 - val_accuracy: 0.6813
Epoch 5/30
22/22 [==============================] - 89s 4s/step - loss: 0.1679 - accuracy: 0.9273 - val_loss: 0.5151 - val_accuracy: 0.6703
Epoch 6/30
22/22 [==============================] - 89s 4s/step - loss: 0.1692 - accuracy: 0.9215 - val_loss: 0.4507 - val_accuracy: 0.7143
Epoch 7/30
22/22 [==============================] - 89s 4s/step - loss: 0.1533 - accuracy: 0.9244 - val_loss: 0.4217 - val_accuracy: 0.7473
Epoch 8/30


In [ ]:
oos_y51 = np.concatenate(oos_y51)
oos_pred51 = np.concatenate(oos_pred51)

In [ ]:
score51 = metrics.accuracy_score(oos_y51.argmax(axis=1),	        #ค่า acc
                                             oos_pred51.argmax(axis=1) )
print(f"Fold score (accuracy): {score51}")

print(classification_report(oos_y51.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        oos_pred51.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

Fold score (accuracy): 0.9312638580931264
              precision    recall  f1-score   support

  Lungcancer       0.99      0.85      0.92       150
      Normal       0.99      0.95      0.97       151
Tuberculosis       0.84      0.99      0.91       150

    accuracy                           0.93       451
   macro avg       0.94      0.93      0.93       451
weighted avg       0.94      0.93      0.93       451



## 6) K-fold  -- DenseNet201

### -->ACC = 0.88
### -->เวลา - วินาที

*   BS = 32
*   fix weight & bias in layer 0-500
*   activation="softmax"
*   learning_rate=0.0001
*   epoch 30

In [ ]:
train_img_Dense = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (224, 224))                                       #ขนาดของ input
    train_img_Dense.append(img)

data_Dense = np.array(train_img_Dense, dtype="float32") / 255.0

100%|██████████| 451/451 [00:05<00:00, 86.49it/s] 


In [ ]:
labels_Dense = np.array(data_train3[['NameType']] )        
labels_Dense.shape = (len(labels_Dense),) 

le_Dense = LabelEncoder()                                                                     
labels_Dense = le_Dense.fit_transform(labels_Dense)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels_Dense = to_categorical(labels_Dense, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

In [ ]:
INIT_LR = 1e-1
BS32 = 32
EPOCHS = 50

aug3 = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.1,
		height_shift_range=0.1,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
base_model_Dense = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
len(base_model_Dense.layers)

707

In [ ]:
base_model_Dense.trainable = True
for layer in base_model_Dense.layers[:500]:    #fix w & bias in layer 0-500
    layer.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Dense.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Dense3 = tf.keras.models.Model(inputs=base_model_Dense.input, outputs=prediction_layer)

model_Dense3.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
kf_Dense3 = KFold(5, shuffle=True, random_state=40) 

oos_y3 = []
oos_pred3 = []
fold = 0

for train, test in kf_Dense3.split(data_Dense):
    fold+=1
    print(f"Fold #{fold}")
        
    x_train_Dense3 = data_Dense[train]
    y_train_Dense3 = labels_Dense[train]
    x_test_Dense3 = data_Dense[test]
    y_test_Dense3 = labels_Dense[test]

    H_Dense3 = model_Dense3.fit( x=aug3.flow(x_train_Dense3, y_train_Dense3, batch_size=BS32),
	                                            validation_data=(x_test_Dense3, y_test_Dense3),
	                                            steps_per_epoch=len(x_train_Dense3) // BS32,
	                                            epochs=30,  callbacks = [callback])
    
    pred_Dense3 = model_Dense3.predict(x_test_Dense3)
    
    oos_y3.append(y_test_Dense3)
    oos_pred3.append(pred_Dense3)  
    print(classification_report(y_test_Dense3.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Dense3.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

#   score = metrics.accuracy_score(y_test3_Incep.argmax(axis=1),	        #ค่า acc
#                                               pred3_Incep.argmax(axis=1)
#    print(f"Fold score (accuracy): {score}")

#    print(confusion_matrix(testY3_Incep.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_Incep.argmax(axis=1) ) )

Fold #1
Epoch 1/30
11/11 [==============================] - 140s 11s/step - loss: 0.5279 - accuracy: 0.6128 - val_loss: 0.4957 - val_accuracy: 0.6044
Epoch 2/30
11/11 [==============================] - 117s 11s/step - loss: 0.3433 - accuracy: 0.7988 - val_loss: 0.5024 - val_accuracy: 0.6044
Epoch 3/30
11/11 [==============================] - 117s 11s/step - loss: 0.3004 - accuracy: 0.8201 - val_loss: 0.4961 - val_accuracy: 0.5824
Epoch 4/30
11/11 [==============================] - 120s 11s/step - loss: 0.2424 - accuracy: 0.8811 - val_loss: 0.4854 - val_accuracy: 0.6154
Epoch 5/30
11/11 [==============================] - 128s 12s/step - loss: 0.2054 - accuracy: 0.9176 - val_loss: 0.4746 - val_accuracy: 0.6264
Epoch 6/30
11/11 [==============================] - 121s 11s/step - loss: 0.1721 - accuracy: 0.9146 - val_loss: 0.4974 - val_accuracy: 0.6044
Epoch 7/30
11/11 [==============================] - 121s 11s/step - loss: 0.1818 - accuracy: 0.9055 - val_loss: 0.5492 - val_accuracy: 0.626

In [ ]:
oos_y3 = np.concatenate(oos_y3)
oos_pred3 = np.concatenate(oos_pred3)

In [ ]:
score = metrics.accuracy_score(oos_y3.argmax(axis=1),	        #ค่า acc
                                             oos_pred3.argmax(axis=1) )
print(f"Fold score (accuracy): {score}")

print(classification_report(oos_y3.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        oos_pred3.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

Fold score (accuracy): 0.8780487804878049
              precision    recall  f1-score   support

  Lungcancer       0.97      0.70      0.81       150
      Normal       0.89      1.00      0.94       151
Tuberculosis       0.81      0.93      0.87       150

    accuracy                           0.88       451
   macro avg       0.89      0.88      0.87       451
weighted avg       0.89      0.88      0.87       451



## 6.1) K-fold  -- DenseNet201

### -->ACC = 0.83 
### -->เวลา 3717 วินาที

*   BS = 32
*   fix weight & bias in layer 0-500
*   activation="sigmoid"
*   learning_rate=0.0001
*   epoch 30

In [ ]:
train_img_Dense = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (224, 224))                                       #ขนาดของ input
    train_img_Dense.append(img)

data_Dense = np.array(train_img_Dense, dtype="float32") / 255.0

100%|██████████| 451/451 [00:05<00:00, 75.37it/s] 


In [ ]:
labels_Dense = np.array(data_train3[['NameType']] )        
labels_Dense.shape = (len(labels_Dense),) 

le_Dense = LabelEncoder()                                                                     
labels_Dense = le_Dense.fit_transform(labels_Dense)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels_Dense = to_categorical(labels_Dense, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

In [ ]:
INIT_LR = 1e-1
BS32 = 32


aug61 = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.1,
		height_shift_range=0.1,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
base_model_Dense = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
len(base_model_Dense.layers)

707

In [ ]:
base_model_Dense.trainable = True
for layer in base_model_Dense.layers[:500]:    #fix w & bias in layer 0-500
    layer.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Dense.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="sigmoid")(average_pooling_layer)   # unit 3
model_Dense61 = tf.keras.models.Model(inputs=base_model_Dense.input, outputs=prediction_layer)

model_Dense61.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
kf_Dense61 = KFold(5, shuffle=True, random_state=40) 

oos_y61 = []
oos_pred61 = []
fold = 0

for train, test in kf_Dense61.split(data_Dense):
    fold+=1
    print(f"Fold #{fold}")
        
    x_train_Dense61 = data_Dense[train]
    y_train_Dense61 = labels_Dense[train]
    x_test_Dense61 = data_Dense[test]
    y_test_Dense61 = labels_Dense[test]

    H_Dense61 = model_Dense61.fit( x=aug61.flow(x_train_Dense61, y_train_Dense61, batch_size=BS32),
	                                            validation_data=(x_test_Dense61, y_test_Dense61),
	                                            steps_per_epoch=len(x_train_Dense61) // BS32,
	                                            epochs=30,  callbacks = [callback])
    
    pred_Dense61 = model_Dense61.predict(x_test_Dense61)
    
    oos_y61.append(y_test_Dense61)
    oos_pred61.append(pred_Dense61)  
    print(classification_report(y_test_Dense61.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Dense61.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

#   score = metrics.accuracy_score(y_test3_Incep.argmax(axis=1),	        #ค่า acc
#                                               pred3_Incep.argmax(axis=1)
#    print(f"Fold score (accuracy): {score}")

#    print(confusion_matrix(testY3_Incep.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_Incep.argmax(axis=1) ) )

Fold #1
Epoch 1/30
11/11 [==============================] - 157s 13s/step - loss: 0.5284 - accuracy: 0.6707 - val_loss: 0.5973 - val_accuracy: 0.5934
Epoch 2/30
11/11 [==============================] - 123s 11s/step - loss: 0.3653 - accuracy: 0.7988 - val_loss: 0.5900 - val_accuracy: 0.5934
Epoch 3/30
11/11 [==============================] - 124s 11s/step - loss: 0.2948 - accuracy: 0.8659 - val_loss: 0.5982 - val_accuracy: 0.5714
Epoch 4/30
11/11 [==============================] - 123s 11s/step - loss: 0.2640 - accuracy: 0.8567 - val_loss: 0.5836 - val_accuracy: 0.5824
              precision    recall  f1-score   support

  Lungcancer       0.50      0.04      0.07        28
      Normal       0.60      1.00      0.75        33
Tuberculosis       0.56      0.63      0.59        30

    accuracy                           0.58        91
   macro avg       0.55      0.56      0.47        91
weighted avg       0.56      0.58      0.49        91

Fold #2
Epoch 1/30
11/11 [=================

In [ ]:
oos_y61 = np.concatenate(oos_y61)
oos_pred61 = np.concatenate(oos_pred61)

In [ ]:
score61 = metrics.accuracy_score(oos_y61.argmax(axis=1),	        #ค่า acc
                                             oos_pred61.argmax(axis=1) )
print(f"Fold score (accuracy): {score61}")

print(classification_report(oos_y61.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        oos_pred61.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

Fold score (accuracy): 0.8337028824833703
              precision    recall  f1-score   support

  Lungcancer       0.98      0.61      0.75       150
      Normal       0.83      0.98      0.90       151
Tuberculosis       0.76      0.91      0.83       150

    accuracy                           0.83       451
   macro avg       0.86      0.83      0.83       451
weighted avg       0.86      0.83      0.83       451



## 7) K-fold  -- DenseNet201

### -->ACC = 0.80
### -->เวลา 6780 วินาที

*   BS = 1
*   fix weight & bias in layer 0-500
*   activation="softmax"
*   learning_rate=0.0001
*   epoch 30

In [ ]:
train_img_Dense = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (224, 224))                                       #ขนาดของ input
    train_img_Dense.append(img)

data_Dense = np.array(train_img_Dense, dtype="float32") / 255.0

100%|██████████| 451/451 [02:25<00:00,  3.10it/s]


In [ ]:
labels_Dense = np.array(data_train3[['NameType']] )        
labels_Dense.shape = (len(labels_Dense),) 

le_Dense = LabelEncoder()                                                                     
labels_Dense = le_Dense.fit_transform(labels_Dense)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels_Dense = to_categorical(labels_Dense, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

In [ ]:
INIT_LR = 1e-1
BS = 1
EPOCHS = 50

aug7 = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.1,
		height_shift_range=0.1,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
base_model_Dense = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
len(base_model_Dense.layers)

74850304/74836368 [==============================] - 1s 0us/step


707

In [ ]:
base_model_Dense.trainable = True
for layer in base_model_Dense.layers[:500]:    #fix w & bias in layer 0-500
    layer.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Dense.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Dense7 = tf.keras.models.Model(inputs=base_model_Dense.input, outputs=prediction_layer)

model_Dense7.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
kf_Dense7 = KFold(5, shuffle=True, random_state=40) 

oos_y7 = []
oos_pred7 = []
fold = 0

for train, test in kf_Dense7.split(data_Dense):
    fold+=1
    print(f"Fold #{fold}")
        
    x_train_Dense7 = data_Dense[train]
    y_train_Dense7 = labels_Dense[train]
    x_test_Dense7 = data_Dense[test]
    y_test_Dense7 = labels_Dense[test]

    H_Dense7 = model_Dense7.fit( x=aug7.flow(x_train_Dense7, y_train_Dense7, batch_size=BS),
	                                            validation_data=(x_test_Dense7, y_test_Dense7),
	                                            steps_per_epoch=len(x_train_Dense7) // BS,
	                                            epochs=30,  callbacks = [callback])
    
    pred_Dense7 = model_Dense7.predict(x_test_Dense7)
    
    oos_y7.append(y_test_Dense7)
    oos_pred7.append(pred_Dense7)  
    print(classification_report(y_test_Dense7.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Dense7.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

#   score = metrics.accuracy_score(y_test3_Incep.argmax(axis=1),	        #ค่า acc
#                                               pred3_Incep.argmax(axis=1)
#    print(f"Fold score (accuracy): {score}")

#    print(confusion_matrix(testY3_Incep.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_Incep.argmax(axis=1) ) )

Fold #1
Epoch 1/30
360/360 [==============================] - 189s 467ms/step - loss: 0.6206 - accuracy: 0.4361 - val_loss: 0.5362 - val_accuracy: 0.6593
Epoch 2/30
360/360 [==============================] - 213s 592ms/step - loss: 0.5373 - accuracy: 0.6139 - val_loss: 0.4485 - val_accuracy: 0.6264
Epoch 3/30
360/360 [==============================] - 210s 585ms/step - loss: 0.4510 - accuracy: 0.6944 - val_loss: 0.3502 - val_accuracy: 0.7473
Epoch 4/30
360/360 [==============================] - 216s 599ms/step - loss: 0.3795 - accuracy: 0.7417 - val_loss: 0.3338 - val_accuracy: 0.7582
Epoch 5/30
360/360 [==============================] - 220s 612ms/step - loss: 0.3188 - accuracy: 0.8083 - val_loss: 0.3708 - val_accuracy: 0.7143
Epoch 6/30
360/360 [==============================] - 220s 612ms/step - loss: 0.2769 - accuracy: 0.8444 - val_loss: 0.4142 - val_accuracy: 0.7143
Epoch 7/30
360/360 [==============================] - 220s 610ms/step - loss: 0.2481 - accuracy: 0.8417 - val_loss: 

In [ ]:
oos_y7 = np.concatenate(oos_y7)
oos_pred7 = np.concatenate(oos_pred7)

In [ ]:
score7 = metrics.accuracy_score(oos_y7.argmax(axis=1),	        #ค่า acc
                                             oos_pred7.argmax(axis=1) )
print(f"Fold score (accuracy): {score7}")

print(classification_report(oos_y7.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        oos_pred7.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

Fold score (accuracy): 0.8048780487804879
              precision    recall  f1-score   support

  Lungcancer       0.79      0.81      0.80       150
      Normal       0.98      0.80      0.88       151
Tuberculosis       0.70      0.81      0.75       150

    accuracy                           0.80       451
   macro avg       0.82      0.80      0.81       451
weighted avg       0.82      0.80      0.81       451



## 7.1) K-fold  -- DenseNet201

### -->ACC = 0.79
### -->เวลา 9506.36 วินาที

*   BS = 1
*   fix weight & bias in layer 0-500
*   activation="sigmoid"
*   learning_rate=0.0001
*   epoch 30

In [ ]:
train_img_Dense = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (224, 224))                                       #ขนาดของ input
    train_img_Dense.append(img)

data_Dense = np.array(train_img_Dense, dtype="float32") / 255.0

100%|██████████| 451/451 [01:44<00:00,  4.30it/s]


In [ ]:
labels_Dense = np.array(data_train3[['NameType']] )        
labels_Dense.shape = (len(labels_Dense),) 

le_Dense = LabelEncoder()                                                                     
labels_Dense = le_Dense.fit_transform(labels_Dense)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels_Dense = to_categorical(labels_Dense, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

In [ ]:
INIT_LR = 1e-1
BS = 1
EPOCHS = 50

aug71 = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.1,
		height_shift_range=0.1,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
base_model_Dense = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
len(base_model_Dense.layers)

74850304/74836368 [==============================] - 0s 0us/step


707

In [ ]:
base_model_Dense.trainable = True
for layer in base_model_Dense.layers[:500]:    #fix w & bias in layer 0-500
    layer.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Dense.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="sigmoid")(average_pooling_layer)   # unit 3
model_Dense71 = tf.keras.models.Model(inputs=base_model_Dense.input, outputs=prediction_layer)

model_Dense71.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
kf_Dense71 = KFold(5, shuffle=True, random_state=40) 

oos_y71 = []
oos_pred71 = []
fold = 0

for train, test in kf_Dense71.split(data_Dense):
    fold+=1
    print(f"Fold #{fold}")
        
    x_train_Dense71 = data_Dense[train]
    y_train_Dense71 = labels_Dense[train]
    x_test_Dense71 = data_Dense[test]
    y_test_Dense71 = labels_Dense[test]

    H_Dense71 = model_Dense71.fit( x=aug71.flow(x_train_Dense71, y_train_Dense71, batch_size=BS),
	                                            validation_data=(x_test_Dense71, y_test_Dense71),
	                                            steps_per_epoch=len(x_train_Dense71) // BS,
	                                            epochs=30,  callbacks = [callback])
    
    pred_Dense71 = model_Dense71.predict(x_test_Dense71)
    
    oos_y71.append(y_test_Dense71)
    oos_pred71.append(pred_Dense71)  
    print(classification_report(y_test_Dense71.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Dense71.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

#   score = metrics.accuracy_score(y_test3_Incep.argmax(axis=1),	        #ค่า acc
#                                               pred3_Incep.argmax(axis=1)
#    print(f"Fold score (accuracy): {score}")

#    print(confusion_matrix(testY3_Incep.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_Incep.argmax(axis=1) ) )

Fold #1
Epoch 1/30
360/360 [==============================] - 191s 474ms/step - loss: 0.6221 - accuracy: 0.4750 - val_loss: 0.5166 - val_accuracy: 0.6923
Epoch 2/30
360/360 [==============================] - 322s 894ms/step - loss: 0.5358 - accuracy: 0.6278 - val_loss: 0.4582 - val_accuracy: 0.6813
Epoch 3/30
360/360 [==============================] - 331s 919ms/step - loss: 0.4576 - accuracy: 0.6417 - val_loss: 0.3911 - val_accuracy: 0.7582
Epoch 4/30
360/360 [==============================] - 331s 920ms/step - loss: 0.3793 - accuracy: 0.7778 - val_loss: 0.3635 - val_accuracy: 0.7692
Epoch 5/30
360/360 [==============================] - 343s 953ms/step - loss: 0.3443 - accuracy: 0.7750 - val_loss: 0.4369 - val_accuracy: 0.7253
Epoch 6/30
360/360 [==============================] - 354s 984ms/step - loss: 0.3001 - accuracy: 0.8250 - val_loss: 0.4009 - val_accuracy: 0.7363
Epoch 7/30
360/360 [==============================] - 334s 928ms/step - loss: 0.2655 - accuracy: 0.8306 - val_loss: 

In [ ]:
oos_y71 = np.concatenate(oos_y71)
oos_pred71 = np.concatenate(oos_pred71)

In [ ]:
score71 = metrics.accuracy_score(oos_y71.argmax(axis=1),	        #ค่า acc
                                             oos_pred71.argmax(axis=1) )
print(f"Fold score (accuracy): {score71}")

print(classification_report(oos_y71.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        oos_pred71.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

Fold score (accuracy): 0.7915742793791575
              precision    recall  f1-score   support

  Lungcancer       0.88      0.82      0.85       150
      Normal       1.00      0.66      0.79       151
Tuberculosis       0.64      0.90      0.75       150

    accuracy                           0.79       451
   macro avg       0.84      0.79      0.80       451
weighted avg       0.84      0.79      0.80       451



# Fine tuning 0-400

## 4) K-fold  -- DenseNet201

### -->ACC = 
### -->เวลา - วินาที

*   BS = 8
*   fix weight & bias in layer 0-400
*   activation="softmax"
*   learning_rate=0.0001
*   epoch 30

In [31]:
train_img_Dense = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (224, 224))                                       #ขนาดของ input
    train_img_Dense.append(img)

data_Dense = np.array(train_img_Dense, dtype="float32") / 255.0

  0%|          | 0/451 [00:00<?, ?it/s]


error: ignored

In [ ]:
labels_Dense = np.array(data_train3[['NameType']] )        
labels_Dense.shape = (len(labels_Dense),) 

le_Dense = LabelEncoder()                                                                     
labels_Dense = le_Dense.fit_transform(labels_Dense)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels_Dense = to_categorical(labels_Dense, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

In [ ]:
INIT_LR = 1e-1
BS = 8
EPOCHS = 50

aug = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.1,
		height_shift_range=0.1,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
base_model_Dense = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
len(base_model_Dense.layers)

In [ ]:
base_model_Dense.trainable = True
for layer in base_model_Dense.layers[:400]:    #fix w & bias in layer 0-400
    layer.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Dense.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Dense = tf.keras.models.Model(inputs=base_model_Dense.input, outputs=prediction_layer)

model_Dense.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
kf_Dense = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Dense.split(data_Dense):
    fold+=1
    print(f"Fold #{fold}")
        
    x_train_Dense = data_Dense[train]
    y_train_Dense = labels_Dense[train]
    x_test_Dense = data_Dense[test]
    y_test_Dense = labels_Dense[test]

    H_Dense = model_Dense.fit( x=aug.flow(x_train_Dense, y_train_Dense, batch_size=BS),
	                                            validation_data=(x_test_Dense, y_test_Dense),
	                                            steps_per_epoch=len(x_train_Dense) // BS,
	                                            epochs=30,  callbacks = [callback])
    
    pred_Dense = model_Dense.predict(x_test_Dense)
    
    oos_y.append(y_test_Dense)
    oos_pred.append(pred_Dense)  
    print(classification_report(y_test_Dense.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Dense.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

#   score = metrics.accuracy_score(y_test3_Incep.argmax(axis=1),	        #ค่า acc
#                                               pred3_Incep.argmax(axis=1)
#    print(f"Fold score (accuracy): {score}")

#    print(confusion_matrix(testY3_Incep.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_Incep.argmax(axis=1) ) )

In [ ]:
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)

In [ ]:
score = metrics.accuracy_score(oos_y.argmax(axis=1),	        #ค่า acc
                                             oos_pred.argmax(axis=1) )
print(f"Fold score (accuracy): {score}")

print(classification_report(oos_y.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        oos_pred.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

## 4.1) K-fold  -- DenseNet201  

### -->ACC = 0.90
### -->เวลา 4575.96 วินาที

*   BS = 8
*   fix weight & bias in layer 0-500
*   activation="sigmoid"
*   learning_rate=0.0001
*   epoch 30

In [ ]:
train_img_Dense = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (224, 224))                                       #ขนาดของ input
    train_img_Dense.append(img)

data_Dense = np.array(train_img_Dense, dtype="float32") / 255.0

100%|██████████| 451/451 [00:05<00:00, 81.94it/s] 


In [ ]:
labels_Dense = np.array(data_train3[['NameType']] )        
labels_Dense.shape = (len(labels_Dense),) 

le_Dense = LabelEncoder()                                                                     
labels_Dense = le_Dense.fit_transform(labels_Dense)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels_Dense = to_categorical(labels_Dense, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

In [ ]:
INIT_LR = 1e-1
BS8 = 8
EPOCHS = 50

aug41 = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.1,
		height_shift_range=0.1,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
base_model_Dense = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
len(base_model_Dense.layers)

707

In [ ]:
base_model_Dense.trainable = True
for layer in base_model_Dense.layers[:500]:    #fix w & bias in layer 0-500
    layer.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Dense.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="sigmoid")(average_pooling_layer)   # unit 3
model_Dense41 = tf.keras.models.Model(inputs=base_model_Dense.input, outputs=prediction_layer)

model_Dense41.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
kf_Dense41 = KFold(5, shuffle=True, random_state=40) 

oos_y41 = []
oos_pred41 = []
fold = 0

for train, test in kf_Dense41.split(data_Dense):
    fold+=1
    print(f"Fold #{fold}")
        
    x_train_Dense41 = data_Dense[train]
    y_train_Dense41 = labels_Dense[train]
    x_test_Dense41 = data_Dense[test]
    y_test_Dense41 = labels_Dense[test]

    H_Dense41 = model_Dense41.fit( x=aug41.flow(x_train_Dense41, y_train_Dense41, batch_size=BS8),
	                                            validation_data=(x_test_Dense41, y_test_Dense41),
	                                            steps_per_epoch=len(x_train_Dense41) // BS8,
	                                            epochs=30,  callbacks = [callback])
    
    pred_Dense41 = model_Dense41.predict(x_test_Dense41)
    
    oos_y41.append(y_test_Dense41)
    oos_pred41.append(pred_Dense41)  
    print(classification_report(y_test_Dense41.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Dense41.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

#   score = metrics.accuracy_score(y_test3_Incep.argmax(axis=1),	        #ค่า acc
#                                               pred3_Incep.argmax(axis=1)
#    print(f"Fold score (accuracy): {score}")

#    print(confusion_matrix(testY3_Incep.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_Incep.argmax(axis=1) ) )

Fold #1
Epoch 1/30
45/45 [==============================] - 119s 2s/step - loss: 0.5008 - accuracy: 0.6444 - val_loss: 0.5943 - val_accuracy: 0.5604
Epoch 2/30
45/45 [==============================] - 99s 2s/step - loss: 0.3438 - accuracy: 0.7833 - val_loss: 0.5383 - val_accuracy: 0.6154
Epoch 3/30
45/45 [==============================] - 103s 2s/step - loss: 0.2788 - accuracy: 0.8528 - val_loss: 0.4846 - val_accuracy: 0.6154
Epoch 4/30
45/45 [==============================] - 104s 2s/step - loss: 0.2418 - accuracy: 0.8889 - val_loss: 0.4779 - val_accuracy: 0.6593
Epoch 5/30
45/45 [==============================] - 104s 2s/step - loss: 0.2206 - accuracy: 0.8750 - val_loss: 0.3742 - val_accuracy: 0.7253
Epoch 6/30
45/45 [==============================] - 105s 2s/step - loss: 0.1690 - accuracy: 0.9250 - val_loss: 0.5127 - val_accuracy: 0.7143
Epoch 7/30
45/45 [==============================] - 103s 2s/step - loss: 0.1680 - accuracy: 0.9306 - val_loss: 0.6544 - val_accuracy: 0.7143
Epoch 

In [ ]:
oos_y41 = np.concatenate(oos_y41)
oos_pred41 = np.concatenate(oos_pred41)

In [ ]:
score41 = metrics.accuracy_score(oos_y41.argmax(axis=1),	        #ค่า acc
                                             oos_pred41.argmax(axis=1) )
print(f"Fold score (accuracy): {score41}")

print(classification_report(oos_y41.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        oos_pred41.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

Fold score (accuracy): 0.8957871396895787
              precision    recall  f1-score   support

  Lungcancer       0.97      0.83      0.90       150
      Normal       0.99      0.87      0.93       151
Tuberculosis       0.78      0.98      0.87       150

    accuracy                           0.90       451
   macro avg       0.91      0.90      0.90       451
weighted avg       0.91      0.90      0.90       451



## 5) K-fold  -- DenseNet201

### -->ACC = 0.93
### -->เวลา - นาที

*   BS = 16
*   fix weight & bias in layer 0-500
*   activation="softmax"
*   learning_rate=0.0001
*   epoch 30

In [ ]:
train_img_Dense = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (224, 224))                                       #ขนาดของ input
    train_img_Dense.append(img)

data_Dense = np.array(train_img_Dense, dtype="float32") / 255.0

100%|██████████| 451/451 [00:05<00:00, 85.20it/s] 


In [ ]:
labels_Dense = np.array(data_train3[['NameType']] )        
labels_Dense.shape = (len(labels_Dense),) 

le_Dense = LabelEncoder()                                                                     
labels_Dense = le_Dense.fit_transform(labels_Dense)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels_Dense = to_categorical(labels_Dense, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

In [ ]:
INIT_LR = 1e-1
BS16 = 16
EPOCHS = 50

aug2 = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.1,
		height_shift_range=0.1,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
base_model_Dense = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
len(base_model_Dense.layers)

707

In [ ]:
base_model_Dense.trainable = True
for layer in base_model_Dense.layers[:600]:    #fix w & bias in layer 0-600
    layer.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Dense.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Dense2 = tf.keras.models.Model(inputs=base_model_Dense.input, outputs=prediction_layer)

model_Dense2.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
kf_Dense2 = KFold(5, shuffle=True, random_state=40) 

oos_y2 = []
oos_pred2 = []
fold = 0

for train, test in kf_Dense2.split(data_Dense):
    fold+=1
    print(f"Fold #{fold}")
        
    x_train_Dense2 = data_Dense[train]
    y_train_Dense2 = labels_Dense[train]
    x_test_Dense2 = data_Dense[test]
    y_test_Dense2 = labels_Dense[test]

    H_Dense2 = model_Dense2.fit( x=aug2.flow(x_train_Dense2, y_train_Dense2, batch_size=BS16),
	                                            validation_data=(x_test_Dense2, y_test_Dense2),
	                                            steps_per_epoch=len(x_train_Dense2) // BS16,
	                                            epochs=30,  callbacks = [callback])
    
    pred_Dense2 = model_Dense2.predict(x_test_Dense2)
    
    oos_y2.append(y_test_Dense2)
    oos_pred2.append(pred_Dense2)  
    print(classification_report(y_test_Dense2.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Dense2.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

#   score = metrics.accuracy_score(y_test3_Incep.argmax(axis=1),	        #ค่า acc
#                                               pred3_Incep.argmax(axis=1)
#    print(f"Fold score (accuracy): {score}")

#    print(confusion_matrix(testY3_Incep.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_Incep.argmax(axis=1) ) )

Fold #1
Epoch 1/30
22/22 [==============================] - 131s 5s/step - loss: 0.5335 - accuracy: 0.6570 - val_loss: 0.5158 - val_accuracy: 0.6593
Epoch 2/30
22/22 [==============================] - 114s 5s/step - loss: 0.3912 - accuracy: 0.7616 - val_loss: 0.4862 - val_accuracy: 0.6703
Epoch 3/30
22/22 [==============================] - 115s 5s/step - loss: 0.3135 - accuracy: 0.8227 - val_loss: 0.4457 - val_accuracy: 0.7033
Epoch 4/30
22/22 [==============================] - 114s 5s/step - loss: 0.2949 - accuracy: 0.8285 - val_loss: 0.4228 - val_accuracy: 0.6813
Epoch 5/30
22/22 [==============================] - 112s 5s/step - loss: 0.2592 - accuracy: 0.8750 - val_loss: 0.4161 - val_accuracy: 0.7033
Epoch 6/30
22/22 [==============================] - 112s 5s/step - loss: 0.2337 - accuracy: 0.8750 - val_loss: 0.3942 - val_accuracy: 0.7253
Epoch 7/30
22/22 [==============================] - 112s 5s/step - loss: 0.2114 - accuracy: 0.8866 - val_loss: 0.3773 - val_accuracy: 0.7363
Epoch

In [ ]:
oos_y2 = np.concatenate(oos_y2)
oos_pred2 = np.concatenate(oos_pred2)

In [ ]:
score = metrics.accuracy_score(oos_y2.argmax(axis=1),	        #ค่า acc
                                             oos_pred2.argmax(axis=1) )
print(f"Fold score (accuracy): {score}")

print(classification_report(oos_y2.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        oos_pred2.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

Fold score (accuracy): 0.926829268292683
              precision    recall  f1-score   support

  Lungcancer       0.98      0.83      0.90       150
      Normal       0.99      0.97      0.98       151
Tuberculosis       0.83      0.99      0.90       150

    accuracy                           0.93       451
   macro avg       0.94      0.93      0.93       451
weighted avg       0.94      0.93      0.93       451



## 5.1) K-fold  -- DenseNet201

### -->ACC = 0.93
### -->เวลา 4335.68 นาที

*   BS = 16
*   fix weight & bias in layer 0-500
*   activation="sigmoid"
*   learning_rate=0.0001
*   epoch 30

In [ ]:
train_img_Dense = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (224, 224))                                       #ขนาดของ input
    train_img_Dense.append(img)

data_Dense = np.array(train_img_Dense, dtype="float32") / 255.0

100%|██████████| 451/451 [00:05<00:00, 86.62it/s] 


In [ ]:
labels_Dense = np.array(data_train3[['NameType']] )        
labels_Dense.shape = (len(labels_Dense),) 

le_Dense = LabelEncoder()                                                                     
labels_Dense = le_Dense.fit_transform(labels_Dense)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels_Dense = to_categorical(labels_Dense, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

In [ ]:
INIT_LR = 1e-1
BS16 = 16
EPOCHS = 50

aug51 = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.1,
		height_shift_range=0.1,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
base_model_Dense = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
len(base_model_Dense.layers)

707

In [ ]:
base_model_Dense.trainable = True
for layer in base_model_Dense.layers[:500]:    #fix w & bias in layer 0-500
    layer.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Dense.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="sigmoid")(average_pooling_layer)   # unit 3
model_Dense51 = tf.keras.models.Model(inputs=base_model_Dense.input, outputs=prediction_layer)

model_Dense51.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
kf_Dense51 = KFold(5, shuffle=True, random_state=40) 

oos_y51 = []
oos_pred51 = []
fold = 0

for train, test in kf_Dense51.split(data_Dense):
    fold+=1
    print(f"Fold #{fold}")
        
    x_train_Dense51 = data_Dense[train]
    y_train_Dense51 = labels_Dense[train]
    x_test_Dense51 = data_Dense[test]
    y_test_Dense51 = labels_Dense[test]

    H_Dense51 = model_Dense51.fit( x=aug51.flow(x_train_Dense51, y_train_Dense51, batch_size=BS16),
	                                            validation_data=(x_test_Dense51, y_test_Dense51),
	                                            steps_per_epoch=len(x_train_Dense51) // BS16,
	                                            epochs=30,  callbacks = [callback])
    
    pred_Dense51 = model_Dense51.predict(x_test_Dense51)
    
    oos_y51.append(y_test_Dense51)
    oos_pred51.append(pred_Dense51)  
    print(classification_report(y_test_Dense51.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Dense51.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

#   score = metrics.accuracy_score(y_test3_Incep.argmax(axis=1),	        #ค่า acc
#                                               pred3_Incep.argmax(axis=1)
#    print(f"Fold score (accuracy): {score}")

#    print(confusion_matrix(testY3_Incep.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_Incep.argmax(axis=1) ) )

Fold #1
Epoch 1/30
22/22 [==============================] - 113s 4s/step - loss: 0.4910 - accuracy: 0.6818 - val_loss: 0.6110 - val_accuracy: 0.5495
Epoch 2/30
22/22 [==============================] - 91s 4s/step - loss: 0.3051 - accuracy: 0.8285 - val_loss: 0.6054 - val_accuracy: 0.5385
Epoch 3/30
22/22 [==============================] - 89s 4s/step - loss: 0.2485 - accuracy: 0.8692 - val_loss: 0.5594 - val_accuracy: 0.5714
Epoch 4/30
22/22 [==============================] - 89s 4s/step - loss: 0.2070 - accuracy: 0.8895 - val_loss: 0.4729 - val_accuracy: 0.6813
Epoch 5/30
22/22 [==============================] - 89s 4s/step - loss: 0.1679 - accuracy: 0.9273 - val_loss: 0.5151 - val_accuracy: 0.6703
Epoch 6/30
22/22 [==============================] - 89s 4s/step - loss: 0.1692 - accuracy: 0.9215 - val_loss: 0.4507 - val_accuracy: 0.7143
Epoch 7/30
22/22 [==============================] - 89s 4s/step - loss: 0.1533 - accuracy: 0.9244 - val_loss: 0.4217 - val_accuracy: 0.7473
Epoch 8/30


In [ ]:
oos_y51 = np.concatenate(oos_y51)
oos_pred51 = np.concatenate(oos_pred51)

In [ ]:
score51 = metrics.accuracy_score(oos_y51.argmax(axis=1),	        #ค่า acc
                                             oos_pred51.argmax(axis=1) )
print(f"Fold score (accuracy): {score51}")

print(classification_report(oos_y51.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        oos_pred51.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

Fold score (accuracy): 0.9312638580931264
              precision    recall  f1-score   support

  Lungcancer       0.99      0.85      0.92       150
      Normal       0.99      0.95      0.97       151
Tuberculosis       0.84      0.99      0.91       150

    accuracy                           0.93       451
   macro avg       0.94      0.93      0.93       451
weighted avg       0.94      0.93      0.93       451



## 6) K-fold  -- DenseNet201

### -->ACC = 0.88
### -->เวลา - วินาที

*   BS = 32
*   fix weight & bias in layer 0-500
*   activation="softmax"
*   learning_rate=0.0001
*   epoch 30

In [ ]:
train_img_Dense = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (224, 224))                                       #ขนาดของ input
    train_img_Dense.append(img)

data_Dense = np.array(train_img_Dense, dtype="float32") / 255.0

100%|██████████| 451/451 [00:05<00:00, 86.49it/s] 


In [ ]:
labels_Dense = np.array(data_train3[['NameType']] )        
labels_Dense.shape = (len(labels_Dense),) 

le_Dense = LabelEncoder()                                                                     
labels_Dense = le_Dense.fit_transform(labels_Dense)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels_Dense = to_categorical(labels_Dense, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

In [ ]:
INIT_LR = 1e-1
BS32 = 32
EPOCHS = 50

aug3 = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.1,
		height_shift_range=0.1,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
base_model_Dense = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
len(base_model_Dense.layers)

707

In [ ]:
base_model_Dense.trainable = True
for layer in base_model_Dense.layers[:500]:    #fix w & bias in layer 0-500
    layer.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Dense.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Dense3 = tf.keras.models.Model(inputs=base_model_Dense.input, outputs=prediction_layer)

model_Dense3.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
kf_Dense3 = KFold(5, shuffle=True, random_state=40) 

oos_y3 = []
oos_pred3 = []
fold = 0

for train, test in kf_Dense3.split(data_Dense):
    fold+=1
    print(f"Fold #{fold}")
        
    x_train_Dense3 = data_Dense[train]
    y_train_Dense3 = labels_Dense[train]
    x_test_Dense3 = data_Dense[test]
    y_test_Dense3 = labels_Dense[test]

    H_Dense3 = model_Dense3.fit( x=aug3.flow(x_train_Dense3, y_train_Dense3, batch_size=BS32),
	                                            validation_data=(x_test_Dense3, y_test_Dense3),
	                                            steps_per_epoch=len(x_train_Dense3) // BS32,
	                                            epochs=30,  callbacks = [callback])
    
    pred_Dense3 = model_Dense3.predict(x_test_Dense3)
    
    oos_y3.append(y_test_Dense3)
    oos_pred3.append(pred_Dense3)  
    print(classification_report(y_test_Dense3.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Dense3.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

#   score = metrics.accuracy_score(y_test3_Incep.argmax(axis=1),	        #ค่า acc
#                                               pred3_Incep.argmax(axis=1)
#    print(f"Fold score (accuracy): {score}")

#    print(confusion_matrix(testY3_Incep.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_Incep.argmax(axis=1) ) )

Fold #1
Epoch 1/30
11/11 [==============================] - 140s 11s/step - loss: 0.5279 - accuracy: 0.6128 - val_loss: 0.4957 - val_accuracy: 0.6044
Epoch 2/30
11/11 [==============================] - 117s 11s/step - loss: 0.3433 - accuracy: 0.7988 - val_loss: 0.5024 - val_accuracy: 0.6044
Epoch 3/30
11/11 [==============================] - 117s 11s/step - loss: 0.3004 - accuracy: 0.8201 - val_loss: 0.4961 - val_accuracy: 0.5824
Epoch 4/30
11/11 [==============================] - 120s 11s/step - loss: 0.2424 - accuracy: 0.8811 - val_loss: 0.4854 - val_accuracy: 0.6154
Epoch 5/30
11/11 [==============================] - 128s 12s/step - loss: 0.2054 - accuracy: 0.9176 - val_loss: 0.4746 - val_accuracy: 0.6264
Epoch 6/30
11/11 [==============================] - 121s 11s/step - loss: 0.1721 - accuracy: 0.9146 - val_loss: 0.4974 - val_accuracy: 0.6044
Epoch 7/30
11/11 [==============================] - 121s 11s/step - loss: 0.1818 - accuracy: 0.9055 - val_loss: 0.5492 - val_accuracy: 0.626

In [ ]:
oos_y3 = np.concatenate(oos_y3)
oos_pred3 = np.concatenate(oos_pred3)

In [ ]:
score = metrics.accuracy_score(oos_y3.argmax(axis=1),	        #ค่า acc
                                             oos_pred3.argmax(axis=1) )
print(f"Fold score (accuracy): {score}")

print(classification_report(oos_y3.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        oos_pred3.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

Fold score (accuracy): 0.8780487804878049
              precision    recall  f1-score   support

  Lungcancer       0.97      0.70      0.81       150
      Normal       0.89      1.00      0.94       151
Tuberculosis       0.81      0.93      0.87       150

    accuracy                           0.88       451
   macro avg       0.89      0.88      0.87       451
weighted avg       0.89      0.88      0.87       451



## 6.1) K-fold  -- DenseNet201

### -->ACC = 0.83 
### -->เวลา 3717 วินาที

*   BS = 32
*   fix weight & bias in layer 0-500
*   activation="sigmoid"
*   learning_rate=0.0001
*   epoch 30

In [ ]:
train_img_Dense = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (224, 224))                                       #ขนาดของ input
    train_img_Dense.append(img)

data_Dense = np.array(train_img_Dense, dtype="float32") / 255.0

100%|██████████| 451/451 [00:05<00:00, 75.37it/s] 


In [ ]:
labels_Dense = np.array(data_train3[['NameType']] )        
labels_Dense.shape = (len(labels_Dense),) 

le_Dense = LabelEncoder()                                                                     
labels_Dense = le_Dense.fit_transform(labels_Dense)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels_Dense = to_categorical(labels_Dense, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

In [ ]:
INIT_LR = 1e-1
BS32 = 32


aug61 = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.1,
		height_shift_range=0.1,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
base_model_Dense = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
len(base_model_Dense.layers)

707

In [ ]:
base_model_Dense.trainable = True
for layer in base_model_Dense.layers[:500]:    #fix w & bias in layer 0-500
    layer.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Dense.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="sigmoid")(average_pooling_layer)   # unit 3
model_Dense61 = tf.keras.models.Model(inputs=base_model_Dense.input, outputs=prediction_layer)

model_Dense61.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
kf_Dense61 = KFold(5, shuffle=True, random_state=40) 

oos_y61 = []
oos_pred61 = []
fold = 0

for train, test in kf_Dense61.split(data_Dense):
    fold+=1
    print(f"Fold #{fold}")
        
    x_train_Dense61 = data_Dense[train]
    y_train_Dense61 = labels_Dense[train]
    x_test_Dense61 = data_Dense[test]
    y_test_Dense61 = labels_Dense[test]

    H_Dense61 = model_Dense61.fit( x=aug61.flow(x_train_Dense61, y_train_Dense61, batch_size=BS32),
	                                            validation_data=(x_test_Dense61, y_test_Dense61),
	                                            steps_per_epoch=len(x_train_Dense61) // BS32,
	                                            epochs=30,  callbacks = [callback])
    
    pred_Dense61 = model_Dense61.predict(x_test_Dense61)
    
    oos_y61.append(y_test_Dense61)
    oos_pred61.append(pred_Dense61)  
    print(classification_report(y_test_Dense61.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Dense61.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

#   score = metrics.accuracy_score(y_test3_Incep.argmax(axis=1),	        #ค่า acc
#                                               pred3_Incep.argmax(axis=1)
#    print(f"Fold score (accuracy): {score}")

#    print(confusion_matrix(testY3_Incep.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_Incep.argmax(axis=1) ) )

Fold #1
Epoch 1/30
11/11 [==============================] - 157s 13s/step - loss: 0.5284 - accuracy: 0.6707 - val_loss: 0.5973 - val_accuracy: 0.5934
Epoch 2/30
11/11 [==============================] - 123s 11s/step - loss: 0.3653 - accuracy: 0.7988 - val_loss: 0.5900 - val_accuracy: 0.5934
Epoch 3/30
11/11 [==============================] - 124s 11s/step - loss: 0.2948 - accuracy: 0.8659 - val_loss: 0.5982 - val_accuracy: 0.5714
Epoch 4/30
11/11 [==============================] - 123s 11s/step - loss: 0.2640 - accuracy: 0.8567 - val_loss: 0.5836 - val_accuracy: 0.5824
              precision    recall  f1-score   support

  Lungcancer       0.50      0.04      0.07        28
      Normal       0.60      1.00      0.75        33
Tuberculosis       0.56      0.63      0.59        30

    accuracy                           0.58        91
   macro avg       0.55      0.56      0.47        91
weighted avg       0.56      0.58      0.49        91

Fold #2
Epoch 1/30
11/11 [=================

In [ ]:
oos_y61 = np.concatenate(oos_y61)
oos_pred61 = np.concatenate(oos_pred61)

In [ ]:
score61 = metrics.accuracy_score(oos_y61.argmax(axis=1),	        #ค่า acc
                                             oos_pred61.argmax(axis=1) )
print(f"Fold score (accuracy): {score61}")

print(classification_report(oos_y61.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        oos_pred61.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

Fold score (accuracy): 0.8337028824833703
              precision    recall  f1-score   support

  Lungcancer       0.98      0.61      0.75       150
      Normal       0.83      0.98      0.90       151
Tuberculosis       0.76      0.91      0.83       150

    accuracy                           0.83       451
   macro avg       0.86      0.83      0.83       451
weighted avg       0.86      0.83      0.83       451



## 7) K-fold  -- DenseNet201

### -->ACC = 0.80
### -->เวลา 6780 วินาที

*   BS = 1
*   fix weight & bias in layer 0-500
*   activation="softmax"
*   learning_rate=0.0001
*   epoch 30

In [ ]:
train_img_Dense = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (224, 224))                                       #ขนาดของ input
    train_img_Dense.append(img)

data_Dense = np.array(train_img_Dense, dtype="float32") / 255.0

100%|██████████| 451/451 [02:25<00:00,  3.10it/s]


In [ ]:
labels_Dense = np.array(data_train3[['NameType']] )        
labels_Dense.shape = (len(labels_Dense),) 

le_Dense = LabelEncoder()                                                                     
labels_Dense = le_Dense.fit_transform(labels_Dense)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels_Dense = to_categorical(labels_Dense, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

In [ ]:
INIT_LR = 1e-1
BS = 1
EPOCHS = 50

aug7 = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.1,
		height_shift_range=0.1,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
base_model_Dense = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
len(base_model_Dense.layers)

74850304/74836368 [==============================] - 1s 0us/step


707

In [ ]:
base_model_Dense.trainable = True
for layer in base_model_Dense.layers[:500]:    #fix w & bias in layer 0-500
    layer.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Dense.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Dense7 = tf.keras.models.Model(inputs=base_model_Dense.input, outputs=prediction_layer)

model_Dense7.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
kf_Dense7 = KFold(5, shuffle=True, random_state=40) 

oos_y7 = []
oos_pred7 = []
fold = 0

for train, test in kf_Dense7.split(data_Dense):
    fold+=1
    print(f"Fold #{fold}")
        
    x_train_Dense7 = data_Dense[train]
    y_train_Dense7 = labels_Dense[train]
    x_test_Dense7 = data_Dense[test]
    y_test_Dense7 = labels_Dense[test]

    H_Dense7 = model_Dense7.fit( x=aug7.flow(x_train_Dense7, y_train_Dense7, batch_size=BS),
	                                            validation_data=(x_test_Dense7, y_test_Dense7),
	                                            steps_per_epoch=len(x_train_Dense7) // BS,
	                                            epochs=30,  callbacks = [callback])
    
    pred_Dense7 = model_Dense7.predict(x_test_Dense7)
    
    oos_y7.append(y_test_Dense7)
    oos_pred7.append(pred_Dense7)  
    print(classification_report(y_test_Dense7.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Dense7.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

#   score = metrics.accuracy_score(y_test3_Incep.argmax(axis=1),	        #ค่า acc
#                                               pred3_Incep.argmax(axis=1)
#    print(f"Fold score (accuracy): {score}")

#    print(confusion_matrix(testY3_Incep.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_Incep.argmax(axis=1) ) )

Fold #1
Epoch 1/30
360/360 [==============================] - 189s 467ms/step - loss: 0.6206 - accuracy: 0.4361 - val_loss: 0.5362 - val_accuracy: 0.6593
Epoch 2/30
360/360 [==============================] - 213s 592ms/step - loss: 0.5373 - accuracy: 0.6139 - val_loss: 0.4485 - val_accuracy: 0.6264
Epoch 3/30
360/360 [==============================] - 210s 585ms/step - loss: 0.4510 - accuracy: 0.6944 - val_loss: 0.3502 - val_accuracy: 0.7473
Epoch 4/30
360/360 [==============================] - 216s 599ms/step - loss: 0.3795 - accuracy: 0.7417 - val_loss: 0.3338 - val_accuracy: 0.7582
Epoch 5/30
360/360 [==============================] - 220s 612ms/step - loss: 0.3188 - accuracy: 0.8083 - val_loss: 0.3708 - val_accuracy: 0.7143
Epoch 6/30
360/360 [==============================] - 220s 612ms/step - loss: 0.2769 - accuracy: 0.8444 - val_loss: 0.4142 - val_accuracy: 0.7143
Epoch 7/30
360/360 [==============================] - 220s 610ms/step - loss: 0.2481 - accuracy: 0.8417 - val_loss: 

In [ ]:
oos_y7 = np.concatenate(oos_y7)
oos_pred7 = np.concatenate(oos_pred7)

In [ ]:
score7 = metrics.accuracy_score(oos_y7.argmax(axis=1),	        #ค่า acc
                                             oos_pred7.argmax(axis=1) )
print(f"Fold score (accuracy): {score7}")

print(classification_report(oos_y7.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        oos_pred7.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

Fold score (accuracy): 0.8048780487804879
              precision    recall  f1-score   support

  Lungcancer       0.79      0.81      0.80       150
      Normal       0.98      0.80      0.88       151
Tuberculosis       0.70      0.81      0.75       150

    accuracy                           0.80       451
   macro avg       0.82      0.80      0.81       451
weighted avg       0.82      0.80      0.81       451



## 7.1) K-fold  -- DenseNet201

### -->ACC = 0.79
### -->เวลา 9506.36 วินาที

*   BS = 1
*   fix weight & bias in layer 0-500
*   activation="sigmoid"
*   learning_rate=0.0001
*   epoch 30

In [ ]:
train_img_Dense = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (224, 224))                                       #ขนาดของ input
    train_img_Dense.append(img)

data_Dense = np.array(train_img_Dense, dtype="float32") / 255.0

100%|██████████| 451/451 [01:44<00:00,  4.30it/s]


In [ ]:
labels_Dense = np.array(data_train3[['NameType']] )        
labels_Dense.shape = (len(labels_Dense),) 

le_Dense = LabelEncoder()                                                                     
labels_Dense = le_Dense.fit_transform(labels_Dense)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels_Dense = to_categorical(labels_Dense, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

In [ ]:
INIT_LR = 1e-1
BS = 1
EPOCHS = 50

aug71 = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.1,
		height_shift_range=0.1,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
base_model_Dense = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
len(base_model_Dense.layers)

74850304/74836368 [==============================] - 0s 0us/step


707

In [ ]:
base_model_Dense.trainable = True
for layer in base_model_Dense.layers[:500]:    #fix w & bias in layer 0-500
    layer.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Dense.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="sigmoid")(average_pooling_layer)   # unit 3
model_Dense71 = tf.keras.models.Model(inputs=base_model_Dense.input, outputs=prediction_layer)

model_Dense71.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
kf_Dense71 = KFold(5, shuffle=True, random_state=40) 

oos_y71 = []
oos_pred71 = []
fold = 0

for train, test in kf_Dense71.split(data_Dense):
    fold+=1
    print(f"Fold #{fold}")
        
    x_train_Dense71 = data_Dense[train]
    y_train_Dense71 = labels_Dense[train]
    x_test_Dense71 = data_Dense[test]
    y_test_Dense71 = labels_Dense[test]

    H_Dense71 = model_Dense71.fit( x=aug71.flow(x_train_Dense71, y_train_Dense71, batch_size=BS),
	                                            validation_data=(x_test_Dense71, y_test_Dense71),
	                                            steps_per_epoch=len(x_train_Dense71) // BS,
	                                            epochs=30,  callbacks = [callback])
    
    pred_Dense71 = model_Dense71.predict(x_test_Dense71)
    
    oos_y71.append(y_test_Dense71)
    oos_pred71.append(pred_Dense71)  
    print(classification_report(y_test_Dense71.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Dense71.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

#   score = metrics.accuracy_score(y_test3_Incep.argmax(axis=1),	        #ค่า acc
#                                               pred3_Incep.argmax(axis=1)
#    print(f"Fold score (accuracy): {score}")

#    print(confusion_matrix(testY3_Incep.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_Incep.argmax(axis=1) ) )

Fold #1
Epoch 1/30
360/360 [==============================] - 191s 474ms/step - loss: 0.6221 - accuracy: 0.4750 - val_loss: 0.5166 - val_accuracy: 0.6923
Epoch 2/30
360/360 [==============================] - 322s 894ms/step - loss: 0.5358 - accuracy: 0.6278 - val_loss: 0.4582 - val_accuracy: 0.6813
Epoch 3/30
360/360 [==============================] - 331s 919ms/step - loss: 0.4576 - accuracy: 0.6417 - val_loss: 0.3911 - val_accuracy: 0.7582
Epoch 4/30
360/360 [==============================] - 331s 920ms/step - loss: 0.3793 - accuracy: 0.7778 - val_loss: 0.3635 - val_accuracy: 0.7692
Epoch 5/30
360/360 [==============================] - 343s 953ms/step - loss: 0.3443 - accuracy: 0.7750 - val_loss: 0.4369 - val_accuracy: 0.7253
Epoch 6/30
360/360 [==============================] - 354s 984ms/step - loss: 0.3001 - accuracy: 0.8250 - val_loss: 0.4009 - val_accuracy: 0.7363
Epoch 7/30
360/360 [==============================] - 334s 928ms/step - loss: 0.2655 - accuracy: 0.8306 - val_loss: 

In [ ]:
oos_y71 = np.concatenate(oos_y71)
oos_pred71 = np.concatenate(oos_pred71)

In [ ]:
score71 = metrics.accuracy_score(oos_y71.argmax(axis=1),	        #ค่า acc
                                             oos_pred71.argmax(axis=1) )
print(f"Fold score (accuracy): {score71}")

print(classification_report(oos_y71.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        oos_pred71.argmax(axis=1),  
                                        target_names=le_Dense.classes_))

Fold score (accuracy): 0.7915742793791575
              precision    recall  f1-score   support

  Lungcancer       0.88      0.82      0.85       150
      Normal       1.00      0.66      0.79       151
Tuberculosis       0.64      0.90      0.75       150

    accuracy                           0.79       451
   macro avg       0.84      0.79      0.80       451
weighted avg       0.84      0.79      0.80       451

